# Load Packages

In [1]:
import yfinance as yf
import pandas as pd
#import cryptocompare
import numpy as np
import os

from openpyxl import Workbook
from openpyxl.drawing.image import Image
from itertools import combinations
import requests
from datetime import datetime, timedelta

import ccxt

from tabulate import tabulate

import matplotlib.pyplot as plt
from collections import defaultdict

import seaborn as sns
from typing import Dict, List, Tuple, Optional
import warnings

Coinbase BTC Premium Index: https://www.coinglass.com/pro/i/coinbase-bitcoin-premium-index

BTC Liquidation Map: https://www.coinglass.com/pro/futures/LiquidationMap

Perp futures to spot volume: https://www.coinglass.com/pro/perpteual-spot-volume

BTC 25 Delta Skew: https://www.theblock.co/data/crypto-markets/options/btc-option-skew-delta-25

Crypto Greed Fear Index: https://alternative.me/crypto/fear-and-greed-index/

Volmex BTC Volatility: https://charts.volmex.finance/symbol/BVIV

Funding Rate Heatmap: https://www.coinglass.com/FundingRateHeatMap

Liquidations Ratio and OI/Market Cap from Coinalyze.net

Bitcoin Dominance Chart: https://in.tradingview.com/chart/7cGHhC4a/?symbol=CRYPTOCAP%3ABTC.D

Meme Marketcap Chart: https://in.tradingview.com/chart/7cGHhC4a/?symbol=CRYPTOCAP%3AMEME.C

# Sentiment Tracker

### Messari Sentiment Signals
https://messari.io/signals?filter=curated&change=absolute&hide=wrapped-coins,stablecoins

# Themes Tracker

### Daily performance

In [ ]:
## Set the dates
start_dt = "2024-12-31"  ## yyyy-mm-dd format
end_dt = "2025-11-10"

In [ ]:
## Read the list of coins and themes

# Path to the Excel file
excel_file_path = r'C:\Users\jhuku\OneDrive\Documents\2. Crypto Research\Crypto Portfolio\Themes_mapping.xlsx'

# Read the Excel file into a DataFrame
df = pd.read_excel(excel_file_path)


## Extract the 'symbol' column and save it to the theme_list and theme column to themes
theme_list = df['Symbol'].tolist()
themes = df['Theme'].tolist()

In [ ]:
# Map to Binance trading pairs
binance_pairs = [f"{sym}/USDT" for sym in theme_list]

# Filter valid pairs
exchange = ccxt.kucoin()   ## binance was not working hence used kucoin
markets = exchange.load_markets()
available_pairs = set(markets.keys())
valid_pairs = [pair for pair in binance_pairs if pair in available_pairs]

In [ ]:
len(valid_pairs)

In [ ]:
# Fetch daily close prices
timeframe = '1d'
limit = 90
sleep_seconds = 0.2

import time

frames = []

for pair in valid_pairs:
    try:
        base = pair.split('/')[0]  # e.g. BTC
        ohlcv = exchange.fetch_ohlcv(pair, timeframe=timeframe, limit=limit)
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df = df[['timestamp', 'close']].rename(columns={'close': base})
        frames.append(df.set_index('timestamp'))
        time.sleep(sleep_seconds)
    except Exception as e:
        print(f"Error fetching {pair}: {e}")

# Merge all into wide format
price_theme = pd.concat(frames, axis=1).sort_index()

In [ ]:
price_theme.head()

In [ ]:
# Only show columns with at least one null - CHECK to see if there any null
null_columns = price_theme.columns[price_theme.isnull().any()]
print(null_columns)

In [ ]:
# Drop columns where all rows are NaN
price_theme = price_theme.dropna(axis=1, how='any')

In [ ]:
# Only show columns with at least one null - CHECK to see if there any null
null_columns = price_theme.columns[price_theme.isnull().any()]
print(null_columns)

In [ ]:
### Calculate Returns over Different Lookback periods

# Lookback periods (in days)
lookbacks = [1, 3, 5, 10, 15, 30, 60]

# Choose a specific date to calculate returns up to (e.g. latest date)
target_date = price_theme.index[-1]

# Prepare output
results = {}

In [ ]:
for lb in lookbacks:
    current = price_theme.loc[target_date]
    past_date = target_date - pd.Timedelta(days=lb)

    # Find the closest available past date
    if past_date in price_theme.index:
        past = price_theme.loc[past_date]
    else:
        # Use the most recent available date before target
        past = price_theme[:target_date].iloc[-lb]

    returns = (current - past) / past
    results[f'{lb}d'] = returns

In [ ]:
# Final output
returns_df = pd.DataFrame(results)
returns_df.index.name = 'Coin'

In [ ]:
# Create a mapping from original ticker to theme (w/o deletions)
ticker_to_theme = dict(zip(theme_list, themes))
remaining_tickers = returns_df.T.columns.to_list()

In [ ]:
# Now map themes for remaining tickers
theme_values = [ticker_to_theme[ticker] for ticker in remaining_tickers]

In [ ]:
## Adding back the Theme column to the dataframe
returns_df['Theme'] = theme_values

In [ ]:
returns_df['Coin'] = returns_df.index  ## Adding the names of Coins as a column

In [ ]:
returns_df.head()

In [ ]:
#### Calculate Excess Return (vs. group avg) for each coin

# Check for duplicates in the data first
dupes = returns_df[returns_df.duplicated(subset=['Coin', 'Theme'], keep=False)]
print(dupes)

## Remove Duplicates
returns_df = returns_df.drop_duplicates(subset=['Coin', 'Theme'])

In [ ]:
# STEP 1: Base setup - calculating average return for each theme & excess return of each coin vs. category average
lookback_cols = ['1d', '3d', '5d', '10d', '15d', '30d', '60d']

df_long = returns_df.melt(id_vars=['Coin', 'Theme'], value_vars=lookback_cols,
                          var_name='Period', value_name='Return')

df_long['Theme_Avg'] = df_long.groupby(['Theme', 'Period'])['Return'].transform('median')
df_long['Excess_Return'] = df_long['Return'] - df_long['Theme_Avg']

returns_wide = df_long.pivot(index=['Coin', 'Theme'], columns='Period', values='Return')
excess_wide = df_long.pivot(index=['Coin', 'Theme'], columns='Period', values='Excess_Return')

returns_wide.columns = [f"{col}" for col in returns_wide.columns]
excess_wide.columns = [f"{col}_Excess" for col in excess_wide.columns]

final_df = pd.concat([returns_wide, excess_wide], axis=1)

In [ ]:
# STEP 2: Multiply by 100 and round
final_df = final_df * 100
final_df = final_df.round(2)

# STEP 3: Reorder columns
sorted_cols = []
for col in lookback_cols:
    sorted_cols.append(col)
    sorted_cols.append(f"{col}_Excess")
final_df = final_df[sorted_cols]

# STEP 4: Reset and sort by Theme + Coin
final_df = final_df.reset_index()
final_df = final_df.sort_values(by=['Theme', 'Coin'])

In [ ]:
# STEP 5: Add Theme Average Rows with Coin = '<Theme>_average'
# For each theme, Excess return is calculated vs. Global Average

# Theme-wise average returns
theme_avg_returns = returns_df.groupby('Theme')[lookback_cols].mean() * 100
theme_avg_returns = theme_avg_returns.round(2)

# Global average returns
global_avg = returns_df[lookback_cols].mean() * 100
global_avg = global_avg.round(2)

# Excess = Theme avg - Global avg
theme_excess = theme_avg_returns.subtract(global_avg, axis=1)
theme_excess.columns = [f"{col}_Excess" for col in theme_excess.columns]

# Combine
theme_combined = pd.concat([theme_avg_returns, theme_excess], axis=1)

# Add 'Coin' = '<Theme>_average'
theme_combined['Coin'] = theme_combined.index + '_average'
theme_combined = theme_combined.reset_index()  # Keeps 'Theme' as a column

# Reorder columns to match final_df
theme_combined = theme_combined[['Coin', 'Theme'] + sorted_cols]

In [ ]:
# STEP 6: Append to final_df
final_df = pd.concat([final_df, theme_combined], ignore_index=True)

# Sort to group properly again
final_df = final_df.sort_values(by=['Theme', 'Coin']).reset_index(drop=True)

In [ ]:
final_df.tail()

In [ ]:
## Save to Excel File on OneDrive
file_path = r'C:\Users\jhuku\OneDrive\Documents\2. Crypto Research\Crypto Portfolio\Output.xlsx'

# Check if the file exists
if not os.path.exists(file_path):
    # File does NOT exist: create new workbook with the first sheet
    with pd.ExcelWriter(file_path, engine='openpyxl', mode='w') as writer:
        final_df.to_excel(writer, sheet_name='ThemeData', index=False)
else:
    # File EXISTS: append or replace sheet as needed
    with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        final_df.to_excel(writer, sheet_name='ThemeData', index=False)

### 1H Performance

In [ ]:
## Read the list of coins and themes

# Path to the Excel file
excel_file_path = r'C:\Users\jhuku\OneDrive\Documents\2. Crypto Research\Crypto Portfolio\Themes_mapping.xlsx'

# Read the Excel file into a DataFrame
df = pd.read_excel(excel_file_path)


## Extract the 'symbol' column and save it to the theme_list and theme column to themes
theme_list = df['Symbol'].tolist()
themes = df['Theme'].tolist()

In [ ]:
## Extract the 'symbol' column and save it to the theme_list and theme column to themes
theme_list = df['Symbol'].tolist()
themes = df['Theme'].tolist()

In [ ]:
# Map to Binance trading pairs
binance_pairs = [f"{sym}/USDT" for sym in theme_list]

# Filter valid pairs
exchange = ccxt.kucoin()   ## binance was not working hence used kucoin
markets = exchange.load_markets()
available_pairs = set(markets.keys())
valid_pairs = [pair for pair in binance_pairs if pair in available_pairs]

In [ ]:
# Fetch hourly close prices
timeframe = '1h'
limit = 24
sleep_seconds = 0.2

import time

frames = []

for pair in valid_pairs:
    try:
        base = pair.split('/')[0]  # e.g. BTC
        ohlcv = exchange.fetch_ohlcv(pair, timeframe=timeframe, limit=limit)
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df = df[['timestamp', 'close']].rename(columns={'close': base})
        frames.append(df.set_index('timestamp'))
        time.sleep(sleep_seconds)
    except Exception as e:
        print(f"Error fetching {pair}: {e}")

# Merge all into wide format
price_theme_h = pd.concat(frames, axis=1).sort_index()

In [ ]:
# Only show columns with at least one null - CHECK to see if there any null
null_columns = price_theme_h.columns[price_theme_h.isnull().any()]
print(null_columns)

In [ ]:
# Drop columns where all rows are NaN
price_theme_h = price_theme_h.dropna(axis=1, how='any')

In [ ]:
# Only show columns with at least one null - CHECK to see if there any null
null_columns = price_theme_h.columns[price_theme_h.isnull().any()]
print(null_columns)

In [ ]:
### Calculate Returns over different time frames/Lookback period 

# Lookback periods (in 15mins)
lookbacks_h = [1, 2, 3, 4, 6]

# Choose a specific date to calculate returns up to (e.g. latest date)
target_date_h = price_theme_h.index[-1]

# Prepare output
results_h = {}


for lb in lookbacks_h:
    current_h = price_theme_h.loc[target_date_h]
    past_date_h = target_date_h - pd.Timedelta(days=lb)

    # Find the closest available past date
    if past_date_h in price_theme_h.index:
        past_h = price_theme_h.loc[past_date_h]
    else:
        # Use the most recent available date before target
        past_h = price_theme_h[:target_date_h].iloc[-lb]

    returns_h = (current_h - past_h) / past_h
    results_h[f'{lb}min'] = returns_h


In [ ]:
# Final output
returns_df_h = pd.DataFrame(results_h)
returns_df_h.index.name = 'Coin'

In [ ]:
# Create a mapping from original ticker to theme (w/o deletions)
ticker_to_theme = dict(zip(theme_list, themes))
remaining_tickers = returns_df_h.T.columns.to_list()

# Now map themes for remaining tickers
theme_values_h = [ticker_to_theme[ticker] for ticker in remaining_tickers]

In [ ]:
# Add as a new row or column
returns_df_h['Theme'] = theme_values_h

returns_df_h['Coin'] = returns_df_h.index  ## Adding the names of Coins as a column

returns_df_h.head()

In [ ]:
#Calculate Excess Return (vs. group avg) for each coin

# STEP 1: Base setup - calculating average return for each theme & excess return of each coin vs. category average
lookback_cols_h = ['1min', '2min', '3min', '4min', '6min']

# Check for duplicates in the data first
dupes = returns_df_h[returns_df_h.duplicated(subset=['Coin', 'Theme'], keep=False)]
print(dupes)

## Remove Duplicates
returns_df_h = returns_df_h.drop_duplicates(subset=['Coin', 'Theme'])


df_long_h = returns_df_h.melt(id_vars=['Coin', 'Theme'], value_vars=lookback_cols_h,
                          var_name='Period', value_name='Return')

df_long_h['Theme_Avg'] = df_long_h.groupby(['Theme', 'Period'])['Return'].transform('median')
df_long_h['Excess_Return'] = df_long_h['Return'] - df_long_h['Theme_Avg']

returns_wide_h = df_long_h.pivot(index=['Coin', 'Theme'], columns='Period', values='Return')
excess_wide_h = df_long_h.pivot(index=['Coin', 'Theme'], columns='Period', values='Excess_Return')

returns_wide_h.columns = [f"{col}" for col in returns_wide_h.columns]
excess_wide_h.columns = [f"{col}_Excess" for col in excess_wide_h.columns]

final_df_h = pd.concat([returns_wide_h, excess_wide_h], axis=1)

In [ ]:
# STEP 2: Multiply by 100 and round
final_df_h = final_df_h * 100
final_df_h = final_df_h.round(2)

# STEP 3: Reorder columns
sorted_cols = []
for col in lookback_cols_h:
    sorted_cols.append(col)
    sorted_cols.append(f"{col}_Excess")
final_df_h = final_df_h[sorted_cols]

# STEP 4: Reset and sort by Theme + Coin
final_df_h = final_df_h.reset_index()
final_df_h = final_df_h.sort_values(by=['Theme', 'Coin'])

In [ ]:
# STEP 5: Add Theme Average Rows with Coin = '<Theme>_average'
# For each theme, Excess return is calculated vs. Global Average

# Theme-wise average returns
theme_avg_returns_h = returns_df_h.groupby('Theme')[lookback_cols_h].mean() * 100
theme_avg_returns_h = theme_avg_returns_h.round(2)

# Global average returns
global_avg_h = returns_df_h[lookback_cols_h].mean() * 100
global_avg_h = global_avg_h.round(2)

# Excess = Theme avg - Global avg
theme_excess_h = theme_avg_returns_h.subtract(global_avg_h, axis=1)
theme_excess_h.columns = [f"{col}_Excess" for col in theme_excess_h.columns]

# Combine
theme_combined_h = pd.concat([theme_avg_returns_h, theme_excess_h], axis=1)

# Add 'Coin' = '<Theme>_average'
theme_combined_h['Coin'] = theme_combined_h.index + '_average'
theme_combined_h = theme_combined_h.reset_index()  # Keeps 'Theme' as a column

# Reorder columns to match final_df_h
theme_combined_h = theme_combined_h[['Coin', 'Theme'] + sorted_cols]


In [ ]:
# STEP 6: Append to final_df_h
final_df_h = pd.concat([final_df_h, theme_combined_h], ignore_index=True)

# Sort to group properly again
final_df_h = final_df_h.sort_values(by=['Theme', 'Coin']).reset_index(drop=True)

In [ ]:
final_df_h.tail()

In [ ]:
## Change the column names to meaningful ones
final_df_h.columns = ['Coin', 'Theme', '1H', '1H_Excess', '2H', '2H_Excess', '3H', '3H_Excess', '4H', '4H_Excess', '6H', '6H_Excess']

In [ ]:
## Save to Excel File on GoogleDrive
file_path = r'C:\Users\jhuku\OneDrive\Documents\2. Crypto Research\Crypto Portfolio\Output.xlsx'

# Check if the file exists
if not os.path.exists(file_path):
    # File does NOT exist: create new workbook with the first sheet
    with pd.ExcelWriter(file_path, engine='openpyxl', mode='w') as writer:
        final_df_h.to_excel(writer, sheet_name='ThemeData_15min', index=False)
else:
    # File EXISTS: append or replace sheet as needed
    with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        final_df_h.to_excel(writer, sheet_name='ThemeData_15min', index=False)

### Themes TS Cum. Perf

In [ ]:
## Use price_theme as price data source as it has data for last 90 days
price_ts = price_theme.copy()

In [ ]:
returns_ts = price_ts.pct_change().dropna()*100

In [ ]:
tickers_ts = returns_ts.columns.values.tolist()

In [ ]:
returns_ts_T = returns_ts.T

In [ ]:
## Adding back the Theme column to the dataframe
returns_ts_T['Theme'] = theme_values

In [ ]:
## Generating dataframes for each theme
def create_theme_dataframes_and_chart(returns_ts_T):
    """
    Create separate dataframes for each theme and generate cumulative returns chart
    """
    
    # Get unique themes
    themes = returns_ts_T['Theme'].unique()
    
    # Get date columns (all columns except 'Theme')
    date_columns = [col for col in returns_ts_T.columns if col != 'Theme']
    
    # Dictionary to store individual theme dataframes
    theme_dataframes = {}
    
    # Create separate dataframe for each theme
    for theme in themes:
        # Filter data for current theme
        theme_data = returns_ts_T[returns_ts_T['Theme'] == theme]
        
        # Calculate median for each date column
        median_values = theme_data[date_columns].median()
        
        # Create dataframe with dates as columns and theme as index
        df_theme = pd.DataFrame([median_values], index=[theme])
        df_theme.columns = date_columns
        
        # Store in dictionary with naming convention df_theme
        theme_dataframes[f'df_{theme.lower().replace(" ", "_")}'] = df_theme
        
   
    # Consolidate all theme dataframes into a single dataframe
    consolidated_df = pd.concat(theme_dataframes.values(), ignore_index=False)
    
 
    # Create cumulative returns chart starting at 100
    # Convert to numeric and handle any non-numeric values
    numeric_df = consolidated_df.apply(pd.to_numeric, errors='coerce')
    
    # Calculate cumulative returns starting at 100
    # Assuming the data represents returns or price changes
    cumulative_returns = pd.DataFrame(index=numeric_df.index, columns=numeric_df.columns)
    
    for theme in numeric_df.index:
        # Convert returns to cumulative returns starting at 100
        theme_data = numeric_df.loc[theme]
        
        # If data represents returns (percentage changes), use cumprod
        # If data represents prices, use direct values
        # Assuming returns data - adjust as needed
        cumulative_returns.loc[theme] = (1 + theme_data/100).cumprod() * 100
    
    return theme_dataframes, consolidated_df, cumulative_returns

In [ ]:
theme_dataframes, consolidated_df, cumulative_returns = create_theme_dataframes_and_chart(returns_ts_T)

In [ ]:
theme_dataframes['df_dex']

In [ ]:
consolidated_df

In [ ]:
cumulative_returns

In [ ]:
cumulative_returns_T = cumulative_returns.T

## Adding the index as a column so that it gets printed in export
cumulative_returns_T['Date'] = cumulative_returns_T.index

In [ ]:
## Save to Excel File on OneDrive
file_path = r'C:\Users\jhuku\OneDrive\Documents\2. Crypto Research\Crypto Portfolio\Output.xlsx'

# Check if the file exists
if not os.path.exists(file_path):
    # File does NOT exist: create new workbook with the first sheet
    with pd.ExcelWriter(file_path, engine='openpyxl', mode='w') as writer:
        cumulative_returns_T.to_excel(writer, sheet_name='ThemeCumPerf', index=False)
else:
    # File EXISTS: append or replace sheet as needed
    with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        cumulative_returns_T.to_excel(writer, sheet_name='ThemeCumPerf', index=False)

# Multi-Strategy Trading Signals

## 4H Data Download

In [ ]:
# Fetch 4H close prices for 180 days
timeframe = '4h'
limit = 1080
sleep_seconds = 0.2

import time

frames = []

for pair in valid_pairs:
    try:
        base = pair.split('/')[0]  # e.g. BTC
        ohlcv = exchange.fetch_ohlcv(pair, timeframe=timeframe, limit=limit)
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df = df[['timestamp', 'close']].rename(columns={'close': base})
        frames.append(df.set_index('timestamp'))
        time.sleep(sleep_seconds)
    except Exception as e:
        print(f"Error fetching {pair}: {e}")

# Merge all into wide format
price_h = pd.concat(frames, axis=1).sort_index()


In [ ]:
# Drop columns where all rows are NaN
price_h = price_h.dropna(axis=1, how='any')

In [ ]:
# Only show columns with at least one null - CHECK to see if there any null
null_columns = price_h.columns[price_h.isnull().any()]
print(null_columns)

In [ ]:
df_h = price_h.copy()

In [ ]:
df_h.shape

In [ ]:
df_h.tail()

## Altcoin Diffusion Index

In [ ]:
def calc_altcoin_diffusion_index(prices, window=180):
    """
    Calculates the altcoin diffusion index over a given rolling window.

    :param prices: pd.DataFrame with columns as coin tickers and dates as index
    :param window: Lookback window in days for measuring outperformance
    :return: pd.Series of diffusion index values
    """
    returns = prices.pct_change(window)
    btc_returns = returns['BTC']

    # Exclude BTC itself for index calculation
    altcoins = [col for col in returns.columns if col != 'BTC']
    outperform = (returns[altcoins].gt(btc_returns, axis=0)).sum(axis=1)
    diffusion_index = 100 * outperform / len(altcoins)
    return diffusion_index


In [ ]:
diffusion_index = calc_altcoin_diffusion_index(df_h, window=90)
diffusion_index.plot(title='Altcoin Diffusion Index')

## Theme Rotation

### Theme Returns

In [ ]:
remaining_tickers_h = df_h.columns.to_list()

In [ ]:
# Now map themes for remaining tickers
theme_values_h = [ticker_to_theme[ticker] for ticker in remaining_tickers_h]

In [ ]:
## Create Theme daily returns using earlier code
price_ts_h = df_h.copy()
returns_ts_h = price_ts_h.pct_change().dropna()*100
tickers_ts_h = returns_ts_h.columns.values.tolist()
returns_ts_h_T = returns_ts_h.T

## Adding back the Theme column to the dataframe
returns_ts_h_T['Theme'] = theme_values_h

In [ ]:
returns_ts_h_T.tail(5)

In [ ]:
## Run the function to get consolidated_df for further use
theme_dataframes_h, consolidated_df_h, cumulative_returns_h = create_theme_dataframes_and_chart(returns_ts_h_T)

In [ ]:
df_theme = consolidated_df_h.T.round(2)

In [ ]:
df_theme.head(5)

### Coins Performance for specific Themes

## Momentum Signals

### Momentum Quality

In [ ]:
### Uses Momentum Quality to filter out coins
### Uses Momentum Quality as defined in: https://alphaarchitect.com/wp-content/uploads/2021/08/The_Quantitative_Momentum_Investing_Philosophy.pdf
### Original paper: https://www3.nd.edu/~zda/Frog.pdf

In [ ]:
## df_h has the 4H price data of the coins
returns_h = df_h.pct_change()

In [ ]:
### Clean the returns to take into account of any 0 values
def clean_returns(returns_h, rolling_window=60):
    cleaned_data = returns_h.copy()
    if (cleaned_data.iloc[0] == 0).all():
        cleaned_data = cleaned_data.iloc[1:].copy()
    for col in cleaned_data.columns:
        zero_mask = cleaned_data[col] == 0
        if zero_mask.sum() > 0:
            rolling_mean = cleaned_data[col].rolling(window=rolling_window, min_periods=1).mean()
            cleaned_data.loc[zero_mask, col] = rolling_mean.loc[zero_mask]
    return cleaned_data

In [ ]:
# Clean the vol_scaled_returns data before processing
returns_cleaned_h = clean_returns(returns_h).dropna()

In [ ]:
### Calculate idmag and remove coins in top 20% of idmag

def compute_idmag_weighted(returns_cleaned_h: pd.DataFrame,
                           lookback: int = 30):
    """
    Compute IDMAG as in Equation (2) of 'Frog in the Pan'.
    """

    weights_by_quintile = {
        0: 5/15,  # Q1 (smallest abs)
        1: 4/15,
        2: 3/15,
        3: 2/15,
        4: 1/15   # Q5 (largest abs)
    }

    idmag_all = {}

    for coin in returns_cleaned_h.columns:
        r = returns_cleaned_h[coin].dropna()

        if len(r) < lookback:
            idmag_all[coin] = np.nan
            continue

        # Rolling implementation
        idmag_values = []

        for i in range(lookback, len(r)):
            window_returns = r.iloc[i - lookback:i]
            abs_returns = window_returns.abs()

            # Rank returns into 5 quintiles (labels 0-4)
            quintile_labels = pd.qcut(abs_returns, 5, labels=False, duplicates='drop')

            # Assign weights according to quintile
            weights = quintile_labels.map(weights_by_quintile)

            # Compute sgn(Return_i) * w_i
            signed_weights = np.sign(window_returns) * weights.values

            # Compute PRET
            pret = window_returns.sum()

            # IDMAG formula
            sgn_pret = np.sign(pret) if pret != 0 else 0
            idmag = - (1 / lookback) * sgn_pret * np.sum(signed_weights)
            idmag_values.append(idmag)

        # Average across all rolling windows
        idmag_all[coin] = np.mean(idmag_values) if len(idmag_values) > 0 else np.nan

    # Create dataframe
    idmag_df = pd.DataFrame([
        {'Name': coin, 'IDMAG': idmag_all[coin]}
        for coin in idmag_all if not pd.isna(idmag_all[coin])
    ])
    idmag_df = idmag_df.sort_values(by='IDMAG', ascending=True)

    # Filter top 80% (drop top 20%)
    cutoff = int(len(idmag_df) * 0.8)
    Mom_Qual = idmag_df.iloc[:cutoff]['Name'].tolist()

    # Filter dataframes
    returns_cleaned_h_filtered = returns_cleaned_h[Mom_Qual]

    return {
        'idmag_df': idmag_df,
        'Mom_Qual': Mom_Qual,
        'returns_cleaned_h_filtered': returns_cleaned_h_filtered,
    }


In [ ]:
result = compute_idmag_weighted(
    returns_cleaned_h=returns_cleaned_h,
    lookback=360  # configurable
)

In [ ]:
idmag_df = result['idmag_df']
Mom_Qual = result['Mom_Qual']
returns_cleaned_h_filtered = result['returns_cleaned_h_filtered']

In [ ]:
# Getting list of coins which had top 20% of idmag and will be removed from further analysis
def get_high_idmag_list(idmag_df: pd.DataFrame, top_pct: float = 0.2):
    """
    Returns list of coin names in the top `top_pct` by IDMAG value.
    """
    top_cutoff = int(len(idmag_df) * top_pct)
    high_idmag_df = idmag_df.iloc[-top_cutoff:]  # Top 20% → end of ascending-sorted df
    High_IDMAG_List = high_idmag_df['Name'].tolist()
    return High_IDMAG_List

In [ ]:
High_IDMAG_List = get_high_idmag_list(idmag_df)

In [ ]:
High_IDMAG_List

### Volatility Scaling

In [ ]:
returns_h = returns_cleaned_h_filtered.copy()

# Calculate standard deviation of returns for each coin over the full period
vol_full_period = returns_h.std()

# Scale each coin's returns by its full-period volatility
vol_scaled_returns = returns_h.divide(vol_full_period, axis='columns')

# Optional cleanup: handle NaN and inf values
vol_scaled_returns = vol_scaled_returns.replace([np.inf, -np.inf], np.nan).fillna(0)

In [ ]:
def clean_vol_scaled_returns(vol_scaled_returns, rolling_window=60):
    cleaned_data = vol_scaled_returns.copy()
    if (cleaned_data.iloc[0] == 0).all():
        cleaned_data = cleaned_data.iloc[1:].copy()
    for col in cleaned_data.columns:
        zero_mask = cleaned_data[col] == 0
        if zero_mask.sum() > 0:
            rolling_mean = cleaned_data[col].rolling(window=rolling_window, min_periods=1).mean()
            cleaned_data.loc[zero_mask, col] = rolling_mean.loc[zero_mask]
        # Remove extreme outliers
        mean_val = cleaned_data[col].mean()
        std_val = cleaned_data[col].std()
        if std_val > 0:
            outlier_mask = np.abs(cleaned_data[col] - mean_val) > 5 * std_val
            cleaned_data.loc[outlier_mask, col] = np.sign(cleaned_data.loc[outlier_mask, col]) * 5 * std_val + mean_val
    return cleaned_data

In [ ]:
# Clean the vol_scaled_returns data before processing
vol_scaled_returns_cleaned = clean_vol_scaled_returns(vol_scaled_returns)

### Excess Momentum across Universe

In [ ]:
## Calculate momentum scores for different lookback periods
def calculate_momentum_scores(vol_scaled_returns, lookback=60, exclude_last=6, winsorize_limit=2):
    shifted_returns = vol_scaled_returns.shift(exclude_last)
    cum_returns = shifted_returns.rolling(window=lookback).sum()
    for col in cum_returns.columns:
        col_data = cum_returns[col].dropna()
        if len(col_data) > 0:
            q5 = col_data.quantile(0.05)
            q95 = col_data.quantile(0.95)
            cum_returns[col] = cum_returns[col].clip(lower=q5, upper=q95)
    z_scores = (cum_returns - cum_returns.mean(axis=1, skipna=True).values[:, None]) / cum_returns.std(axis=1, skipna=True).values[:, None]
    ranks = z_scores.rank(axis=1, ascending=False)
    return z_scores, ranks

In [ ]:
def generate_signals(ranks, top_n=8, bottom_n=8):
    buy_signals_dict = {}
    sell_signals_dict = {}
    for idx in ranks.index:
        row = ranks.loc[idx].dropna()
        if len(row) >= top_n:
            buy_list = row.nsmallest(top_n).index.tolist()
            sell_list = row.nlargest(bottom_n).index.tolist()
            buy_signals_dict[idx] = buy_list
            sell_signals_dict[idx] = sell_list
    buy_signals = pd.Series(buy_signals_dict)
    sell_signals = pd.Series(sell_signals_dict)
    return buy_signals, sell_signals

In [ ]:
## Weekly drawdown limit of 10%

def backtest_with_weekly_drawdown_limit(
    vol_scaled_returns, buy_signals, sell_signals, 
    max_position_size=0.1, stop_loss=-0.05, weekly_drawdown_limit=-0.1
):
    dates = vol_scaled_returns.index
    capital = 1.0
    capital_history = []
    position_returns = []
    date_history = []
    trades_executed = 0
    weekly_returns = []
    periods_per_week = 6 * 7  # 4-hour bars, 6 per day, 7 days

    for i in range(1, len(dates)):
        current_date = dates[i]
        prev_date = dates[i - 1]
        # Update weekly returns window
        if len(weekly_returns) >= periods_per_week:
            weekly_returns.pop(0)
        if len(position_returns) > 0:
            weekly_returns.append(position_returns[-1])
        # Compute rolling weekly capital and drawdown
        weekly_capital = 1.0
        for r in weekly_returns:
            weekly_capital *= (1 + r)
        weekly_drawdown = (weekly_capital - 1.0) / 1.0
        # Enforce weekly drawdown limit
        if weekly_drawdown < weekly_drawdown_limit:
            capital_history.append(capital)
            position_returns.append(0)
            date_history.append(current_date)
            continue
        # Check for valid signals
        if prev_date not in buy_signals.index:
            capital_history.append(capital)
            position_returns.append(0)
            date_history.append(current_date)
            continue
        buy_list = buy_signals.loc[prev_date]
        if not buy_list or len(buy_list) == 0:
            capital_history.append(capital)
            position_returns.append(0)
            date_history.append(current_date)
            continue
        # Calculate returns for the position
        try:
            position_returns_raw = vol_scaled_returns.loc[current_date, buy_list]
            valid_returns = position_returns_raw.dropna()
            if len(valid_returns) == 0:
                daily_returns = 0
            else:
                daily_returns = valid_returns.mean()
            # Apply position sizing
            if abs(daily_returns) > max_position_size:
                daily_returns = np.sign(daily_returns) * max_position_size
            # Apply stop-loss
            if daily_returns < stop_loss:
                daily_returns = stop_loss
            capital *= (1 + daily_returns)
            trades_executed += 1
        except (KeyError, ValueError):
            daily_returns = 0
            capital *= (1 + daily_returns)
        capital_history.append(capital)
        position_returns.append(daily_returns)
        date_history.append(current_date)
    capital_series = pd.Series(capital_history, index=pd.Index(date_history))
    returns_series = pd.Series(position_returns, index=pd.Index(date_history))
    print(f"Total trades executed: {trades_executed}")
    print(f"Final capital: {capital:.4f}")
    if len(capital_history) > 0:
        max_dd = ((min(capital_history) - max(capital_history)) / max(capital_history))
        print(f"Max drawdown: {max_dd:.4f}")
    return capital_series, returns_series

In [ ]:
def plot_results(equity_curve, returns_series, title=""):
    rolling_return = returns_series.rolling(24).mean()  # 4 days
    drawdown = equity_curve / equity_curve.cummax() - 1
    total_return = (equity_curve.iloc[-1] - 1) * 100
    max_dd = drawdown.min() * 100
    returns_std = returns_series.std()
    sharpe = returns_series.mean() / returns_std * np.sqrt(365.25 * 6) if returns_std != 0 else 0
    win_rate = (returns_series > 0).mean() * 100
    fig, axs = plt.subplots(4, 1, figsize=(15, 12))
    axs[0].plot(equity_curve.index, equity_curve.values, label='Equity Curve', linewidth=2)
    axs[0].set_title(f'Equity Curve - {title}\nTotal Return: {total_return:.2f}%, Max DD: {max_dd:.2f}%, Sharpe: {sharpe:.2f}')
    axs[0].legend()
    axs[0].grid(True, alpha=0.3)
    axs[1].plot(rolling_return.index, rolling_return.values, label='Rolling Return (4D)', color='orange')
    axs[1].axhline(y=0, color='black', linestyle='-', alpha=0.3)
    axs[1].set_title('Rolling Return (4 Days)')
    axs[1].legend()
    axs[1].grid(True, alpha=0.3)
    axs[2].fill_between(drawdown.index, drawdown.values, 0, alpha=0.3, color='red')
    axs[2].plot(drawdown.index, drawdown.values, color='red', linewidth=1)
    axs[2].set_title('Drawdown')
    axs[2].set_ylabel('Drawdown %')
    axs[2].grid(True, alpha=0.3)
    axs[3].hist(returns_series.values, bins=50, alpha=0.7, edgecolor='black')
    axs[3].axvline(x=0, color='red', linestyle='--', alpha=0.7)
    axs[3].set_title(f'Returns Distribution - Win Rate: {win_rate:.1f}%')
    axs[3].set_xlabel('Return')
    axs[3].set_ylabel('Frequency')
    axs[3].grid(True, alpha=0.3)
    for ax in axs[:-1]:
        ax.tick_params(axis='x', rotation=45)
    plt.tight_layout()
    plt.show()


In [ ]:
# === MAIN EXECUTION ===
print("Starting momentum strategy with weekly drawdown control...")

vol_scaled_returns_cleaned = clean_vol_scaled_returns(vol_scaled_returns)

## Lookback periods of 10/15/20 days

lookbacks = [60]   
for lb in lookbacks:
    print(f"\n{'='*60}\nTESTING LOOKBACK = {lb}\n{'='*60}")
    z_scores, ranks = calculate_momentum_scores(vol_scaled_returns_cleaned, lookback=lb, exclude_last=6)
    buy_signals, sell_signals = generate_signals(ranks, top_n=8, bottom_n=8)
    if len(buy_signals) == 0:
        print(f"No valid signals generated for lookback {lb}")
        continue
    equity_curve, returns_series = backtest_with_weekly_drawdown_limit(
        vol_scaled_returns_cleaned, buy_signals, sell_signals,
        max_position_size=0.1,  # 10% position size
        stop_loss=-0.05,        # 5% stop loss
        weekly_drawdown_limit=-0.1  # -10% weekly drawdown limit
    )
    if len(equity_curve) > 0:
        plot_results(equity_curve, returns_series, title=f'Lookback = {lb}')
    else:
        print(f"No data to plot for lookback {lb}")


In [ ]:
### Getting the Buy Signals
buy_signals.tail(5)

In [ ]:
### Getting the Sell Signals
sell_signals.tail(5)

In [ ]:
buy_signals.to_excel('buy_signals.xlsx')

In [ ]:
sell_signals.to_excel('sell_signals.xlsx')

In [ ]:
### FUTURE WORK - NEED TO COMBINE SIGNALS FROM DIFFERENT LOOKBACK PERIODS AND TAKE WEIGHTED AVERAGE TO SELECT

### Excess Momentum For each Sector

In [ ]:
## Saving themes of filtered coins to list
filtered_tickers_h = vol_scaled_returns_cleaned.columns.to_list()

In [ ]:
# Now map themes for those tickers
filtered_theme_values_h = [ticker_to_theme[ticker] for ticker in filtered_tickers_h]

In [ ]:
## Adding back the Theme column to the dataframe.T
vol_scaled_returns_cleaned_T = vol_scaled_returns_cleaned.T
vol_scaled_returns_cleaned_T['Theme'] = filtered_theme_values_h

In [ ]:
vol_scaled_returns_cleaned_T.head(5)

In [ ]:
## Creating a list of themes - have removed any themes which do have less than 5 coins manually 
theme_list_short = ['DEX', 'AI', 'DEPIN', 'L2', 'GAMEFI', 'RWA', 'ZKPROOF', 'CROSS_CHAIN', 'MEME', 'LENDING']

In [ ]:
# Create separate datasets for each theme
theme_datasets = {}

for theme in theme_list_short:
    # Filter dataframe for the current theme
    filtered_df = vol_scaled_returns_cleaned_T[vol_scaled_returns_cleaned_T['Theme'] == theme]
    
    # Transpose the filtered dataframe
    transposed_df = filtered_df.T
    
    # Remove the Theme row (since we transposed, Theme is now a row)
    transposed_df = transposed_df.drop('Theme', axis=0)
    
    # Store in dictionary with naming convention
    theme_datasets[f'{theme}_df_T'] = transposed_df
    
    # Optionally, create individual variables for each theme dataset
    globals()[f'{theme}_df_T'] = transposed_df

# Print summary of created datasets
print("Created datasets:")
for theme in theme_list_short:
    dataset_name = f'{theme}_df_T'
    print(f"- {dataset_name}: Shape {theme_datasets[dataset_name].shape}")


In [ ]:
# Access a specific theme dataset
dex_df_T = theme_datasets['DEX_df_T']  # if 'DEX' is in your theme_list_short

In [ ]:
# ===== GENERATE BUY/SELL SIGNALS FOR ALL THEME DATASETS =====
# ===== KEEPING ONLY THE SIGNALS FOR THE LAST DATE =====

# Initialize list to store all signals
all_signals = []

# Process each theme dataset
for theme_name, dataset in theme_datasets.items():
    try:
        # Run momentum analysis on current dataset
        z_scores, ranks = calculate_momentum_scores(dataset, lookback=90, exclude_last=6)
        buy_signals, sell_signals = generate_signals(ranks, top_n=3, bottom_n=3)
        
        # Get signals for the last date only
        if isinstance(buy_signals, pd.DataFrame):
            # If buy_signals is a DataFrame, get the last row
            last_date = buy_signals.index[-1]
            buy_signals = buy_signals.loc[last_date]
        elif isinstance(buy_signals, pd.Series) and len(buy_signals.index) > 1:
            # If it's a Series with multiple dates, get the last one
            buy_signals = buy_signals.iloc[-1] if hasattr(buy_signals.iloc[-1], '__iter__') else [buy_signals.iloc[-1]]
        
        if isinstance(sell_signals, pd.DataFrame):
            # If sell_signals is a DataFrame, get the last row
            last_date = sell_signals.index[-1]
            sell_signals = sell_signals.loc[last_date]
        elif isinstance(sell_signals, pd.Series) and len(sell_signals.index) > 1:
            # If it's a Series with multiple dates, get the last one
            sell_signals = sell_signals.iloc[-1] if hasattr(sell_signals.iloc[-1], '__iter__') else [sell_signals.iloc[-1]]
        
        # Convert signals to comma-separated strings - handle different data types
        # Handle buy signals
        if isinstance(buy_signals, pd.Series):
            buy_signals_str = ', '.join(buy_signals.astype(str)) if not buy_signals.empty else ''
        elif isinstance(buy_signals, list):
            buy_signals_str = ', '.join(buy_signals) if buy_signals else ''
        elif isinstance(buy_signals, (pd.Index, np.ndarray)):
            buy_signals_str = ', '.join(buy_signals.astype(str)) if len(buy_signals) > 0 else ''
        else:
            buy_signals_str = str(buy_signals) if buy_signals is not None else ''
            
        # Handle sell signals
        if isinstance(sell_signals, pd.Series):
            sell_signals_str = ', '.join(sell_signals.astype(str)) if not sell_signals.empty else ''
        elif isinstance(sell_signals, list):
            sell_signals_str = ', '.join(sell_signals) if sell_signals else ''
        elif isinstance(sell_signals, (pd.Index, np.ndarray)):
            sell_signals_str = ', '.join(sell_signals.astype(str)) if len(sell_signals) > 0 else ''
        else:
            sell_signals_str = str(sell_signals) if sell_signals is not None else ''
        
        # Store signals for this theme
        signal_row = {
            'Name': theme_name,
            'Buy': buy_signals_str,
            'Sell': sell_signals_str
        }
        all_signals.append(signal_row)
        
        print(f"✓ Processed {theme_name}: {len(buy_signals) if hasattr(buy_signals, '__len__') else 1} buy, {len(sell_signals) if hasattr(sell_signals, '__len__') else 1} sell signals")
        
    except Exception as e:
        print(f"✗ Error processing {theme_name}: {str(e)}")
        # Add empty row for failed processing
        all_signals.append({
            'Name': theme_name,
            'Buy': '',
            'Sell': ''
        })

# Create consolidated signals dataframe
consolidated_signals = pd.DataFrame(all_signals)


In [ ]:
consolidated_signals

In [ ]:
### FUTURE WORK - NEED TO COMBINE SIGNALS FROM DIFFERENT LOOKBACK PERIODS AND TAKE WEIGHTED AVERAGE TO SELECT

### Volatility Breakout

In [ ]:
import pandas as pd
import numpy as np
from typing import Tuple, Dict, Optional

In [ ]:
class VolatilityBreakoutSignal:
    """
    Volatility breakout signal generator that considers both historical 
    and cross-sectional volatility patterns.
    """
    
    def __init__(self, 
                 historical_window: int = 30,
                 volatility_window: int = 6,
                 cross_sectional_window: int = 20,
                 historical_threshold: float = 2.0,
                 cross_sectional_threshold: float = 1.5):
        """
        Initialize volatility breakout parameters.
        
        Parameters:
        -----------
        historical_window : int
            Lookback period for calculating historical volatility baseline (in 4-hour periods)
        volatility_window : int  
            Rolling window for current volatility calculation (in 4-hour periods)
        cross_sectional_window : int
            Window for cross-sectional volatility comparison
        historical_threshold : float
            Z-score threshold for historical volatility breakout
        cross_sectional_threshold : float
            Z-score threshold for cross-sectional volatility breakout
        """
        self.historical_window = historical_window
        self.volatility_window = volatility_window
        self.cross_sectional_window = cross_sectional_window
        self.historical_threshold = historical_threshold
        self.cross_sectional_threshold = cross_sectional_threshold
    
    def calculate_returns(self, df: pd.DataFrame) -> pd.DataFrame:
        """Calculate log returns from price data."""
        return np.log(df / df.shift(1)).dropna()
    
    def calculate_rolling_volatility(self, returns: pd.DataFrame) -> pd.DataFrame:
        """Calculate rolling volatility (standard deviation of returns)."""
        return returns.rolling(window=self.volatility_window).std()
    
    def calculate_historical_volatility_zscore(self, volatility: pd.DataFrame) -> pd.DataFrame:
        """
        Calculate z-score of current volatility vs historical volatility for each coin.
        """
        historical_mean = volatility.rolling(window=self.historical_window).mean()
        historical_std = volatility.rolling(window=self.historical_window).std()
        
        # Avoid division by zero
        historical_std = historical_std.replace(0, np.nan)
        
        z_scores = (volatility - historical_mean) / historical_std
        return z_scores
    
    def calculate_cross_sectional_zscore(self, volatility: pd.DataFrame) -> pd.DataFrame:
        """
        Calculate z-score of each coin's volatility vs cross-sectional average.
        """
        # Calculate cross-sectional mean and std for each timestamp
        cross_sectional_mean = volatility.mean(axis=1)
        cross_sectional_std = volatility.std(axis=1)
        
        # Broadcast to match original shape
        cs_mean_matrix = np.broadcast_to(cross_sectional_mean.values.reshape(-1, 1), 
                                       volatility.shape)
        cs_std_matrix = np.broadcast_to(cross_sectional_std.values.reshape(-1, 1), 
                                      volatility.shape)
        
        # Create DataFrames with same index/columns as input
        cs_mean_df = pd.DataFrame(cs_mean_matrix, 
                                index=volatility.index, 
                                columns=volatility.columns)
        cs_std_df = pd.DataFrame(cs_std_matrix, 
                               index=volatility.index, 
                               columns=volatility.columns)
        
        # Avoid division by zero
        cs_std_df = cs_std_df.replace(0, np.nan)
        
        # Calculate z-scores
        z_scores = (volatility - cs_mean_df) / cs_std_df
        return z_scores
    
    def calculate_excess_volatility_score(self, 
                                        historical_zscore: pd.DataFrame,
                                        cross_sectional_zscore: pd.DataFrame,
                                        historical_weight: float = 0.6,
                                        cross_sectional_weight: float = 0.4) -> pd.DataFrame:
        """
        Combine historical and cross-sectional z-scores into excess volatility score.
        """
        excess_score = (historical_weight * historical_zscore + 
                       cross_sectional_weight * cross_sectional_zscore)
        return excess_score
    
    def generate_signals(self, df: pd.DataFrame) -> Dict[str, pd.DataFrame]:
        """
        Generate volatility breakout signals and rankings.
        
        Parameters:
        -----------
        df : pd.DataFrame
            Price data with datetime index and coin symbols as columns
            
        Returns:
        --------
        Dict containing:
        - 'signals': Binary signals (1 for breakout, 0 otherwise)
        - 'rankings': Percentile rankings of excess volatility
        - 'excess_volatility': Raw excess volatility scores
        - 'historical_zscore': Historical volatility z-scores
        - 'cross_sectional_zscore': Cross-sectional volatility z-scores
        """
        
        # Step 1: Calculate returns
        returns = self.calculate_returns(df)
        
        # Step 2: Calculate rolling volatility
        volatility = self.calculate_rolling_volatility(returns)
        
        # Step 3: Calculate historical volatility z-scores
        historical_zscore = self.calculate_historical_volatility_zscore(volatility)
        
        # Step 4: Calculate cross-sectional volatility z-scores
        cross_sectional_zscore = self.calculate_cross_sectional_zscore(volatility)
        
        # Step 5: Calculate combined excess volatility score
        excess_volatility = self.calculate_excess_volatility_score(
            historical_zscore, cross_sectional_zscore
        )
        
        # Step 6: Generate binary signals based on thresholds
        historical_signals = (historical_zscore > self.historical_threshold).astype(int)
        cross_sectional_signals = (cross_sectional_zscore > self.cross_sectional_threshold).astype(int)
        
        # Combined signal: either conditions must be met
        combined_signals = (historical_signals | cross_sectional_signals).astype(int)
        
        # Step 7: Calculate rankings (percentile-based)
        rankings = excess_volatility.rank(axis=1, pct=True) * 100
        
        # Step 8: Create strength-based rankings (0-100 scale)
        strength_rankings = pd.DataFrame(index=excess_volatility.index, 
                                       columns=excess_volatility.columns)
        
        for timestamp in excess_volatility.index:
            row_data = excess_volatility.loc[timestamp].dropna()
            if len(row_data) > 0:
                # Rank from highest (strongest) to lowest excess volatility
                ranked = row_data.rank(ascending=False)
                # Convert to 0-100 scale
                max_rank = len(row_data)
                strength_scores = ((max_rank - ranked + 1) / max_rank) * 100
                strength_rankings.loc[timestamp, strength_scores.index] = strength_scores
        
        return {
            'signals': combined_signals,
            'rankings': rankings,
            'strength_rankings': strength_rankings,
            'excess_volatility': excess_volatility,
            'historical_zscore': historical_zscore,
            'cross_sectional_zscore': cross_sectional_zscore,
            'volatility': volatility
        }
    
    def get_top_signals(self, 
                       signals_dict: Dict[str, pd.DataFrame], 
                       timestamp: Optional[str] = None,
                       top_n: int = 10,
                       require_both_signals: bool = True) -> pd.DataFrame:
        """
        Get top N coins with strongest volatility breakout signals.
        
        Parameters:
        -----------
        signals_dict : Dict
            Output from generate_signals()
        timestamp : str, optional
            Specific timestamp to analyze. If None, uses latest.
        top_n : int
            Number of top coins to return
        require_both_signals : bool
            If True, requires both historical and cross-sectional signals.
            If False, returns top coins by excess volatility score regardless of signals.
            
        Returns:
        --------
        DataFrame with top coins and their metrics
        """
        if timestamp is None:
            timestamp = signals_dict['signals'].index[-1]
        
        # Get data for specific timestamp
        signals = signals_dict['signals'].loc[timestamp]
        excess_vol = signals_dict['excess_volatility'].loc[timestamp].dropna()
        strength_rank = signals_dict['strength_rankings'].loc[timestamp]
        hist_z = signals_dict['historical_zscore'].loc[timestamp]
        cs_z = signals_dict['cross_sectional_zscore'].loc[timestamp]
        
        if require_both_signals:
            # Filter for coins with signals (both conditions met)
            signal_coins = signals[signals == 1].index
            
            if len(signal_coins) == 0:
                print(f"No coins meet both thresholds at {timestamp}")
                print("Historical signals:", (signals_dict['historical_zscore'].loc[timestamp] > self.historical_threshold).sum())
                print("Cross-sectional signals:", (signals_dict['cross_sectional_zscore'].loc[timestamp] > self.cross_sectional_threshold).sum())
                return pd.DataFrame()  # No signals
            
            valid_coins = signal_coins
        else:
            # Use all coins with valid excess volatility scores
            valid_coins = excess_vol.index
        
        # Create summary DataFrame
        summary = pd.DataFrame({
            'coin': valid_coins,
            'excess_volatility_score': excess_vol[valid_coins].values,
            'strength_ranking': strength_rank[valid_coins].values,
            'historical_zscore': hist_z[valid_coins].values,
            'cross_sectional_zscore': cs_z[valid_coins].values,
            'has_signal': signals[valid_coins].values if require_both_signals else 'N/A'
        })
        
        # Add individual signal flags for debugging
        hist_signals = (hist_z > self.historical_threshold).astype(int)
        cs_signals = (cs_z > self.cross_sectional_threshold).astype(int)
        
        summary['historical_signal'] = hist_signals[valid_coins].values
        summary['cross_sectional_signal'] = cs_signals[valid_coins].values
        
        # Sort by excess volatility score (descending)
        summary = summary.sort_values('excess_volatility_score', ascending=False)
        
        return summary.head(top_n).reset_index(drop=True)

    def diagnose_signals(self, 
                        signals_dict: Dict[str, pd.DataFrame], 
                        timestamp: Optional[str] = None) -> Dict:
        """
        Diagnose why you might be getting few signals.
        
        Returns statistics about signal generation.
        """
        if timestamp is None:
            timestamp = signals_dict['signals'].index[-1]
        
        hist_z = signals_dict['historical_zscore'].loc[timestamp].dropna()
        cs_z = signals_dict['cross_sectional_zscore'].loc[timestamp].dropna()
        
        hist_signals = (hist_z > self.historical_threshold).sum()
        cs_signals = (cs_z > self.cross_sectional_threshold).sum()
        both_signals = ((hist_z > self.historical_threshold) & 
                       (cs_z > self.cross_sectional_threshold)).sum()
        
        diagnostics = {
            'timestamp': timestamp,
            'total_coins': len(hist_z),
            'historical_threshold': self.historical_threshold,
            'cross_sectional_threshold': self.cross_sectional_threshold,
            'coins_above_historical_threshold': hist_signals,
            'coins_above_cross_sectional_threshold': cs_signals,
            'coins_above_both_thresholds': both_signals,
            'max_historical_zscore': hist_z.max(),
            'max_cross_sectional_zscore': cs_z.max(),
            'mean_historical_zscore': hist_z.mean(),
            'mean_cross_sectional_zscore': cs_z.mean(),
            'std_historical_zscore': hist_z.std(),
            'std_cross_sectional_zscore': cs_z.std()
        }
        
        return diagnostics
    

In [ ]:
# Initialize with your parameters
vb_signal = VolatilityBreakoutSignal(
    historical_window=90,      # 15 days of 4-hour data
    volatility_window=6,       # 1 day rolling volatility
    historical_threshold=1.5,   # 2 std devs above historical
    cross_sectional_threshold=1.0  # 1.5 std devs above peers
)

# Generate all signals and metrics
signals_dict = vb_signal.generate_signals(df_h)

# Get top 11-20 strongest signals
#top_signals = vb_signal.get_top_signals(signals_dict, top_n=10)

top_signals = vb_signal.get_top_signals(signals_dict, top_n=20).iloc[10:20].reset_index(drop=True)


In [ ]:
top_signals

### Price Range Breakout

In [ ]:
# Fetch daily close prices - valid_pairs is defined earlier
timeframe = '1d'
limit = 815 # NEED TO MODIFY - from 1st June 2023 to today
sleep_seconds = 0.2

import time

frames = []

for pair in valid_pairs:
    try:
        base = pair.split('/')[0]  # e.g. BTC
        ohlcv = exchange.fetch_ohlcv(pair, timeframe=timeframe, limit=limit)
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df = df[['timestamp', 'close']].rename(columns={'close': base})
        frames.append(df.set_index('timestamp'))
        time.sleep(sleep_seconds)
    except Exception as e:
        print(f"Error fetching {pair}: {e}")

# Merge all into wide format
price_d = pd.concat(frames, axis=1).sort_index()

In [ ]:
def detect_sr_channels_coins_only(df, 
                                  prd=5,
                                  loopback=200,
                                  min_strength=1,
                                  channel_w_start=0.1,
                                  channel_w_max=0.3,
                                  proximity_start=0.02,
                                  proximity_max=0.1,
                                  top_n=15):

    def detect_once(df, prd, loopback, min_strength, channel_w, proximity):
        results = { "near_support": [], "near_resistance": [],
                    "broke_support": [], "broke_resistance": [] }

        for coin in df.columns:
            prices = df[coin].dropna()
            if len(prices) < loopback + prd + 2:
                continue

            closes = prices.values
            highs, lows = closes, closes

            # Step 1: pivots
            pivots = []
            for i in range(prd, len(closes) - prd):
                is_high = all(highs[i] >= highs[i - k] for k in range(1, prd+1)) and \
                          all(highs[i] >= highs[i + k] for k in range(1, prd+1))
                is_low = all(lows[i] <= lows[i - k] for k in range(1, prd+1)) and \
                         all(lows[i] <= lows[i + k] for k in range(1, prd+1))
                if is_high or is_low:
                    pivots.append((i, closes[i]))

            if not pivots:
                continue

            # Step 2: channels
            max_width = (closes[-loopback:].max() - closes[-loopback:].min()) * channel_w
            channels = []
            for _, val in pivots:
                lo, hi, count = val, val, 0
                for _, val2 in pivots:
                    if abs(val2 - val) <= max_width:
                        lo, hi = min(lo, val2), max(hi, val2)
                        count += 1
                if count >= min_strength:
                    channels.append([lo, hi, count])

            if not channels:
                continue

            # Step 3: score by loopback
            for ch in channels:
                lo, hi, s = ch
                for k in range(1, loopback+1):
                    if len(closes)-k < 0: break
                    if (lows[-k] <= hi and highs[-k] >= lo):
                        ch[2] += 1

            channels = sorted(channels, key=lambda x: x[2], reverse=True)[:3]

            # Step 4: classify
            last, prev = closes[-1], closes[-2]
            support = [c for c in channels if c[0] <= last]
            resistance = [c for c in channels if c[1] >= last]

            if support:
                lo, hi, s = max(support, key=lambda x: x[2])
                if abs(last - lo)/last <= proximity:
                    results["near_support"].append((coin, s))
                if prev >= lo and last < lo:
                    results["broke_support"].append((coin, s))

            if resistance:
                lo, hi, s = max(resistance, key=lambda x: x[2])
                if abs(last - hi)/last <= proximity:
                    results["near_resistance"].append((coin, s))
                if prev <= hi and last > hi:
                    results["broke_resistance"].append((coin, s))

        for k in results:
            results[k] = sorted(results[k], key=lambda x: x[1], reverse=True)

        return results

    # Adaptive loop
    channel_w, proximity = channel_w_start, proximity_start
    while channel_w <= channel_w_max and proximity <= proximity_max:
        res = detect_once(df, prd, loopback, min_strength, channel_w, proximity)

        assigned = set()
        unique_res = {k: [] for k in res}

        # Priority: breakouts first, then near levels
        for key in ["broke_support", "broke_resistance", "near_support", "near_resistance"]:
            for coin, strength in res[key]:
                if coin not in assigned:
                    unique_res[key].append((coin, strength))
                    assigned.add(coin)

        # Limit to top_n and keep only coin names
        for k in unique_res:
            unique_res[k] = [c for c, _ in unique_res[k][:top_n]]

        if any(len(unique_res[k]) > 0 for k in unique_res):
            return unique_res

        # Relax thresholds
        channel_w *= 1.5
        proximity *= 1.5

    return unique_res  # return even if empty


In [ ]:
signals = detect_sr_channels_coins_only(price_d)


In [ ]:
signals['near_support']

In [ ]:
signals['near_resistance']

In [ ]:
signals['broke_support']

In [ ]:
signals['broke_resistance']

### Relative Price Oscillator

In [ ]:
vol_scaled_returns_cleaned_T.head()

In [ ]:
#### NEED TO WORK ON THIS

### Alpha Percentile

In [ ]:
## Checking Alpha of each coin vs BTC and getting %tile ranks for latest obs
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [ ]:
# Fetch 4H close prices for 360 days
timeframe = '1d'
limit = 360
sleep_seconds = 0.2

import time

frames = []

for pair in valid_pairs:
    try:
        base = pair.split('/')[0]  # e.g. BTC
        ohlcv = exchange.fetch_ohlcv(pair, timeframe=timeframe, limit=limit)
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df = df[['timestamp', 'close']].rename(columns={'close': base})
        frames.append(df.set_index('timestamp'))
        time.sleep(sleep_seconds)
    except Exception as e:
        print(f"Error fetching {pair}: {e}")

# Merge all into wide format
df_price_alpha = pd.concat(frames, axis=1).sort_index()

In [ ]:
df_price_alpha.tail()

In [ ]:
# === Step 1: Compute returns ===
df_ret = df_price_alpha.pct_change()

In [ ]:
# === Step 0: Clean df_ret ===
# Drop columns (coins) that are entirely NaN
df_ret = df_ret.dropna(axis=1, how="all").copy()

# Ensure BTC is present
assert "BTC" in df_ret.columns, "BTC column missing"

btc_ret = df_ret["BTC"]

In [ ]:
df_ret.shape

In [ ]:
df_ret.head(5)

In [ ]:
# === Step 1: Function to compute alpha ===
def calc_alpha(coin_window, btc_window):
    mask = (~np.isnan(coin_window)) & (~np.isnan(btc_window))
    if mask.sum() < 30:   # require at least 30 valid obs in 60-day window
        return np.nan
    X = btc_window[mask].reshape(-1, 1)
    y = coin_window[mask].reshape(-1, 1)
    model = LinearRegression().fit(X, y)
    return model.intercept_[0]

In [ ]:
# === Step 2: Compute rolling 90-day alpha for all coins ===
window = 90
alpha_dict = {}

for coin in df_ret.columns:
    if coin == "BTC":
        continue
    values = []
    for i in range(len(df_ret)):
        if i < window - 1:
            values.append(np.nan)
        else:
            coin_window = df_ret[coin].iloc[i-window+1:i+1].values
            btc_window  = btc_ret.iloc[i-window+1:i+1].values
            values.append(calc_alpha(coin_window, btc_window))
    alpha_dict[coin] = values

df_alpha = pd.DataFrame(alpha_dict, index=df_ret.index)

In [ ]:
df_alpha = df_alpha*100
df_alpha = df_alpha.round(2)

In [ ]:
# === Step 3a: Historical percentiles (within-coin over history) ===
df_alpha_hist_pct = df_alpha.apply(lambda x: x.rank(pct=True))

# === Step 3b: Cross-sectional percentiles (within-day across coins) ===
df_alpha_xsec_pct = df_alpha.rank(axis=1, pct=True)

In [ ]:
# === Step 4: Last-day rankings ===
last_hist_pct = df_alpha_hist_pct.iloc[-1].dropna()
last_xsec_pct = df_alpha_xsec_pct.iloc[-1].dropna()

In [ ]:
# Top/bottom 15% (historical)
top_20_hist = last_hist_pct.nlargest(max(1, int(len(last_hist_pct) * 0.20)))
bottom_20_hist = last_hist_pct.nsmallest(max(1, int(len(last_hist_pct) * 0.20)))

# Top/bottom 10% (cross-sectional)
top_20_xsec = last_xsec_pct.nlargest(max(1, int(len(last_xsec_pct) * 0.20)))
bottom_20_xsec = last_xsec_pct.nsmallest(max(1, int(len(last_xsec_pct) * 0.20)))

In [ ]:
top_20_hist[17:]  ## Getting the 2nd decile

In [ ]:
bottom_20_hist[17:]

In [ ]:
top_20_xsec[17:]

In [ ]:
bottom_20_xsec[17:]

In [ ]:
## Plotting rolling alpha for a specfic coin

import matplotlib.pyplot as plt

# Example: pick one or more coins
coins_to_plot = ["BNB"]  # <-- put your coin(s) here

plt.figure(figsize=(12, 6))

for coin in coins_to_plot:
    if coin in df_alpha.columns:
        series = df_alpha[coin].dropna()
        plt.plot(series.index, series.values, label=coin)
    else:
        print(f"⚠️ Coin {coin} not found in df_alpha")

plt.title("Rolling 90-Day Alpha vs BTC")
plt.xlabel("Date")
plt.ylabel("Alpha")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.5)
plt.show()

In [ ]:
### Plotting rolling alpha cross-sectional %tile for the same coin
plt.figure(figsize=(12, 6))

for coin in coins_to_plot:
    if coin in df_alpha_xsec_pct.columns:
        series = df_alpha_xsec_pct[coin].dropna()
        plt.plot(series.index, series.values, label=coin)
    else:
        print(f"⚠️ Coin {coin} not found in df_alpha_xsec_pct")

plt.title("Rolling 90-Day Cross-sectional Alpha %tile vs BTC")
plt.xlabel("Date")
plt.ylabel("Alpha")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.5)
plt.show()

## Trend Following

### Trend Following

In [ ]:
## Filtered list of coins which has been filtered using momentum quality = filtered_tickers_h
## Price dataframe = df_h

In [ ]:
## Keeping only those coins in dataframe which are in the filtered list
df_short_h = df_h[filtered_tickers_h].copy()

In [ ]:
# Parameters
#nw_window = 48.0 # 8 * 6 times/day
#nw_r = 48.0   # 8 * 6 times/day
#nw_start = 150 # 25 * 6 times/day
#ss_wma_period = 90 # 15days * 6times/day
#aema_period = 84 # 14days * 6times/day
#fatl_length = 120 # 20days * 6times/day
#jma_length = 84  # 14days * 6times/day
#phase = 0.5
#hold_days = 18  # 3 days * 6times/day

# Parameters
base_length = 15
nw_start = 150 # 25 * 6 times/day
ss_wma_period = 90 # 15days * 6times/day
aema_period = 6*base_length # 15days * 6times/day
fatl_length = 6*base_length # 15days * 6times/day
jma_length = 6*base_length  # 15days * 6times/day
nw_window = 6*base_length # 14 * 6 times/day
nw_r = 48.0   # 8 * 6 times/day
phase = 0.5
hold_days = 16  

In [ ]:
# Smoothed WMA (LazyLine)
#def smooth_wma(series, length):
#    w2 = int(round(length / 3))
#    w1 = int(round((length - w2) / 2))
#    w3 = int((length - w2) / 2)
#    l1 = series.rolling(w1).mean()
#    l2 = l1.rolling(w2).mean()
#    l3 = l2.rolling(w3).mean()
#    return l3

# Adaptive EMA
def adaptive_ema(series, period):
    ema = series.copy()
    noise = np.zeros_like(series.values)
    for i in range(period, len(series)):
        sig = abs(series.iloc[i] - series.iloc[i - period])
        noise[i] = noise[i - 1] + abs(series.iloc[i] - series.iloc[i - 1]) - abs(series.iloc[i] - series.iloc[i - period])
        noise_val = noise[i] if noise[i] != 0 else 1
        efratio = sig / noise_val
        slow_end = period * 5
        fast_end = max(period / 2.0, 1)
        avg_period = ((sig / noise_val) * (slow_end - fast_end)) + fast_end
        alpha = 2.0 / (1.0 + avg_period)
        ema.iloc[i] = ema.iloc[i - 1] + alpha * (series.iloc[i] - ema.iloc[i - 1])
    return ema

# JFATL
def jfatl(series, fatl_len, jma_len, phase):
    fatl = series.rolling(fatl_len).mean()
    e = 0.5 * (phase + 1)
    wma1 = fatl.rolling(jma_len).mean()
    wma2 = fatl.rolling(jma_len // 2).mean()
    return wma1 * e + wma2 * (1 - e)

# Vectorized Nadaraya-Watson Estimator
def nadaraya_watson_vectorized(series, h, r, start_regression_at_bar):
    n = len(series)
    smoothed = np.full(n, np.nan)
    X = np.arange(n)

    for t in range(start_regression_at_bar, n):
        indices = np.arange(0, t)
        distances = t - indices
        weights = (1 + (distances**2 / ((h**2) * 2 * r))) ** (-r)
        values = series.values[:t]
        smoothed[t] = np.sum(values * weights) / np.sum(weights)

    return pd.Series(smoothed, index=series.index)


In [ ]:
# Indicator Calculation and Signal Generation
indicators = {}
buy_list = []
short_list = []

for coin in df_short_h.columns:
    price = df_short_h[coin]
#    ss_wma = smooth_wma(price, ss_wma_period)
    aema = adaptive_ema(price, aema_period)
    jfatl_val = jfatl(price, fatl_length, jma_length, phase)
    nw_val = nadaraya_watson_vectorized(price, nw_window, nw_r, nw_start)

    indicators[coin] = {
#        "ss_wma": ss_wma,
        "aema": aema,
        "jfatl": jfatl_val,
        "nw": nw_val
    }

# Composite buy/sell logic with whipsaw reduction
#    above = (price > ss_wma) & (price > aema) & (price > jfatl_val) & (price > nw_val)
    above = (price > aema) & (price > jfatl_val) & (price > nw_val)
#    below = (price < ss_wma) & (price < aema) & (price < jfatl_val) & (price < nw_val)
    below = (price < aema) & (price < jfatl_val) & (price < nw_val)

    above_rolling = above.rolling(hold_days).sum()
    below_rolling = below.rolling(hold_days).sum()

    if above_rolling.iloc[-1] == hold_days:
        buy_list.append(coin)
    elif below_rolling.iloc[-1] == hold_days:
        short_list.append(coin)

In [ ]:
buy_list

In [ ]:
short_list

In [ ]:
# Calculate 7-day and 30-day returns
returns_7d = df_short_h.pct_change(42).iloc[-1] * 100
returns_30d = df_short_h.pct_change(180).iloc[-1] * 100

In [ ]:
# Function to generate return tables
def get_return_table(ticker_list, label):
    rows = []
    for ticker in ticker_list:
        if ticker in returns_7d and ticker in returns_30d:
            rows.append([ticker, f"{returns_7d[ticker]:.2f}%", f"{returns_30d[ticker]:.2f}%"])
    if "BTC" in returns_7d:
        rows.append(["BTC", f"{returns_7d['BTC']:.2f}%", f"{returns_30d['BTC']:.2f}%"])
    return tabulate(rows, headers=[f"{label} Ticker", "7-Day Return", "30-Day Return"], tablefmt="pretty")


In [ ]:
buy_return_table = get_return_table(buy_list, "Buy")
short_return_table = get_return_table(short_list, "Short")

In [ ]:
print(buy_return_table)

In [ ]:
print(short_return_table)

### Return-to-trend

In [ ]:
df_short_h = df_h[filtered_tickers_h].copy()

In [ ]:
def return_to_trend_signals_percentile(df_prices, ema_period=50, slope_window=20, 
                                       dev_pct_thresh=0.2, slope_pct_thresh=0.8):
    """
    Adaptive Return-to-Trend Strategy using percentile ranks across coins.
    
    Parameters:
    - df_prices: Price DataFrame with datetime index and coin symbols as columns
    - ema_period: Period for EMA smoothing
    - slope_window: Window to compute EMA slope
    - dev_pct_thresh: Percentile threshold for deviation (e.g., 0.2 = bottom 20%)
    - slope_pct_thresh: Percentile threshold for slope (e.g., 0.8 = top 20%)

    Returns:
    - signals: DataFrame with 1 (buy), -1 (short), 0 (no trade)
    - ema: EMA DataFrame
    - ema_slope: Slope of EMA
    - deviation: % deviation from EMA
    - dev_rank, slope_rank: percentile ranks used for filtering
    """
    # Step 1: Compute EMA
    ema = df_prices.ewm(span=ema_period, adjust=False).mean()

    # Step 2: Compute EMA Slope
    ema_slope = ema.diff(slope_window) / slope_window

    # Step 3: Compute % Deviation
    deviation = (df_prices - ema) / ema

    # Step 4: Compute cross-sectional percentile ranks
    dev_rank = deviation.rank(axis=1, pct=True)
    slope_rank = ema_slope.rank(axis=1, pct=True)

    # Step 5: Signals
    signals = pd.DataFrame(0, index=df_prices.index, columns=df_prices.columns)

    # Buy: pulled back but strong uptrend
    buy_condition = (dev_rank < dev_pct_thresh) & (slope_rank > slope_pct_thresh)
    
    # Sell: extended but strong downtrend
    short_condition = (dev_rank > (1 - dev_pct_thresh)) & (slope_rank < (1 - slope_pct_thresh))

    signals[buy_condition] = 1
    signals[short_condition] = -1

    return signals, ema, ema_slope, deviation, dev_rank, slope_rank


In [ ]:
signals, ema, ema_slope, deviation, dev_rank, slope_rank = return_to_trend_signals_percentile(
    df_short_h,
    ema_period=50,
    slope_window=30,
    dev_pct_thresh=0.3,
    slope_pct_thresh=0.7
)

# Get last row signals
last_row = signals.iloc[-1]
buy_signals = last_row[last_row == 1].index.tolist()
sell_signals = last_row[last_row == -1].index.tolist()

print("Buy signals:", buy_signals)
print("Sell signals:", sell_signals)


| Behavior                | Thresholds                                           |
| ----------------------- | ---------------------------------------------------- |
| More signals            | `dev_pct_thresh = 0.3`, `slope_pct_thresh = 0.7`     |
| Fewer, stronger signals | `dev_pct_thresh = 0.1`, `slope_pct_thresh = 0.9`     |
| Symmetric longs/shorts  | Use same `dev_pct_thresh` and `1 - slope_pct_thresh` |


In [ ]:
### NEED TO WORK ON THIS

### Trend of Trend

In [ ]:

def trend_of_trend_signals(
    df_prices: pd.DataFrame,
    trend_span: int = 55,
    slope_smooth: int = 5,
    accel_smooth: int = 5,
    vol_lookback: int = 120
):
    """
    Compute trend-of-trend components and continuous scores.
    """
    df = df_prices.copy().astype(float)
    df = df.replace([np.inf, -np.inf], np.nan).dropna(how="all")
    df = df.fillna(method="ffill")

    lp = np.log(df)

    # Primary trend
    ema = lp.ewm(span=trend_span, min_periods=trend_span, adjust=False).mean()
    slope = ema.diff().ewm(span=slope_smooth, adjust=False).mean()
    accel = slope.diff().ewm(span=accel_smooth, adjust=False).mean()

    # Vol normalization
    logret = lp.diff()
    vol = logret.rolling(vol_lookback, min_periods=max(20, vol_lookback // 3)).std()

    slope_z = slope / vol
    accel_z = accel / vol

    # Composite score (slope + accel)
    tot_score = 0.6 * slope_z + 0.4 * accel_z

    return {
        "slope_z": slope_z,
        "accel_z": accel_z,
        "tot_score": tot_score
    }

def last_bar_entries_topN(signal_dict, top_n: int = 15):
    """
    Select top N and bottom N coins by tot_score at the last bar.

    Parameters:
      top_n : number of coins to long/short

    Returns:
      timestamp, dict with 'long_entry' and 'short_entry' coin lists
    """
    tot_score = signal_dict["tot_score"]
    last_time = tot_score.index[-1]
    last_scores = tot_score.loc[last_time].dropna()

    if last_scores.empty:
        return last_time, {"long_entry": [], "short_entry": []}

    # Rank by score
    sorted_scores = last_scores.sort_values(ascending=False)

    long_entry = sorted_scores.head(top_n).index.tolist()
    short_entry = sorted_scores.tail(top_n).index.tolist()

    return last_time, {"long_entry": long_entry, "short_entry": short_entry}


In [ ]:
# df_short_h is your wide DataFrame of 4-hourly prices
signals = trend_of_trend_signals(df_short_h)

ts, entries = last_bar_entries_topN(signals, top_n=15)

In [ ]:
print("Long Entry :", entries["long_entry"])

In [ ]:
print("Short Entry:", entries["short_entry"])

## Long/Short

### Long/Short for Each Sector - CODE NOT WORKING

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant
from statsmodels.tsa.stattools import coint

In [ ]:
## Making copy of original price data
df_coint = df_h.copy()

In [ ]:
## Saving prices of filtered coins to list -- filtered coins comes after momentum quality filter
df_coint = df_coint.filter(items=filtered_tickers_h)

In [ ]:
## Adding back the Theme column to the dataframe
df_coint_T = df_coint.T
df_coint_T['Theme'] = filtered_theme_values_h

In [ ]:
## Creating a list of themes - have removed any themes which do have less than 5 coins manually 
coint_list_short = ['DEX', 'AI', 'DEPIN', 'L2', 'GAMEFI', 'RWA', 'ZKPROOF', 'CROSS_CHAIN', 'MEME', 'LENDING']

In [ ]:
# Create separate datasets for each theme
coint_datasets = {}

for theme in coint_list_short:
    # Filter dataframe for the current theme
    filtered_df = df_coint_T[df_coint_T['Theme'] == theme]
    
    # Transpose the filtered dataframe
    transposed_df = filtered_df.T
    
    # Remove the Theme row (since we transposed, Theme is now a row)
    transposed_df = transposed_df.drop('Theme', axis=0)
    
    # Store in dictionary with naming convention
    coint_datasets[f'{theme}_coint_T'] = transposed_df
    
    # Optionally, create individual variables for each theme dataset
    globals()[f'{theme}_coint_T'] = transposed_df

# Print summary of created datasets
print("Created datasets:")
for theme in coint_list_short:
    dataset_name = f'{theme}_coint_T'
    print(f"- {dataset_name}: Shape {coint_datasets[dataset_name].shape}")

In [ ]:
## Creating a list of sector dataframes to look thru later
list_sector_dfs = [DEX_coint_T, AI_coint_T, DEPIN_coint_T, L2_coint_T, GAMEFI_coint_T, RWA_coint_T, ZKPROOF_coint_T,
                   CROSS_CHAIN_coint_T, MEME_coint_T, LENDING_coint_T]


In [ ]:
### NOT WORKING Properly - possibly too many calculations

## Value Buys

### Fresh Momentum

In [ ]:
# Define the fresh momentum function

def fresh_momentum(close_prices, lookback_days=60, holding_period=90):
    returns = close_prices.pct_change(lookback_days).shift(-holding_period)
    # Sum positive returns over holding period and subtract the negative returns
    fm = returns.clip(lower=0).sum() - returns.clip(upper=0).sum()
    return fm

In [ ]:
# Calculate fresh momentum
fresh_momentum_df_ST = pd.DataFrame(index=filtered_tickers_h, columns = ['fresh_mom_ST'])
for ticker in filtered_tickers_h:
    close_prices = df_short_h[ticker]
    fm = fresh_momentum(close_prices)
    fresh_momentum_df_ST.loc[ticker] = fm

In [ ]:
### Distance from ATH and ATL

# Calculate all-time high, low, and associated dates
ath_values = df_short_h.max()
ath_dates = df_short_h.idxmax()

atl_values = df_short_h.min()
atl_dates = df_short_h.idxmin()

In [ ]:
latest_prices = df_short_h.iloc[-1]
ath_diff = ((latest_prices - ath_values) / ath_values) * 100
atl_diff = ((latest_prices - atl_values) / atl_values) * 100

In [ ]:
# Create DataFrames for both ATH and ATL distance
ath_df = pd.DataFrame({
    "ATH Date": ath_dates,
    "Return from ATH (%)": ath_diff
}).sort_values(by="Return from ATH (%)", ascending=False).head(25)

atl_df = pd.DataFrame({
    "ATL Date": atl_dates,
    "Return from ATL (%)": atl_diff
}).sort_values(by="Return from ATL (%)").head(25)

In [ ]:
### NEED TO COMBINE THE DATAFRAMES TOGETHER

## Technical Positioning

### Funding Rates

In [14]:
import requests
import pandas as pd
import time
from datetime import datetime, timedelta
from typing import List, Dict
from scipy.stats import zscore

In [15]:
## Getting Funding rates for last 90 days
class BinanceFundingRateCollector:
    def __init__(self):
        self.base_url = "https://fapi.binance.com"
        self.session = requests.Session()
        self.session.headers.update({
            'Content-Type': 'application/json',
            'User-Agent': 'Mozilla/5.0'
        })

    def get_funding_rate_history(self, symbol: str, start_time: int, end_time: int, limit: int = 1000) -> List[Dict]:
        url = f"{self.base_url}/fapi/v1/fundingRate"
        params = {
            "symbol": symbol,
            "startTime": start_time,
            "endTime": end_time,
            "limit": limit
        }

        try:
            response = self.session.get(url, params=params)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data for {symbol}: {e}")
            return []

    def get_all_funding_rates(self, symbol: str, days_back: int = 30) -> List[Dict]:
        end_time = int(time.time() * 1000)
        start_time = end_time - (days_back * 24 * 60 * 60 * 1000)

        all_data = []
        current_start = start_time

        while current_start < end_time:
            batch_end = min(current_start + (333 * 24 * 60 * 60 * 1000), end_time)
            batch_data = self.get_funding_rate_history(symbol, current_start, batch_end, 1000)

            if not batch_data:
                break

            all_data.extend(batch_data)

            if len(batch_data) < 1000:
                break

            current_start = batch_data[-1]['fundingTime'] + 1
            time.sleep(0.1)

        return all_data

    def process_funding_data(self, raw_data: List[Dict]) -> pd.DataFrame:
        if not raw_data:
            return pd.DataFrame()

        df = pd.DataFrame(raw_data)
        df['fundingTime'] = pd.to_datetime(df['fundingTime'], unit='ms')
        df['fundingRate'] = df['fundingRate'].astype(float)
        df = df.sort_values('fundingTime').reset_index(drop=True)
        df['fundingRatePercent'] = df['fundingRate'] * 100
        df['annualizedRate'] = df['fundingRate'] * 365 * 3
        df['annualizedRatePercent'] = df['annualizedRate'] * 100

        return df

    def get_active_futures_symbols(self) -> List[str]:
        url = f"{self.base_url}/fapi/v1/exchangeInfo"
        try:
            response = self.session.get(url)
            response.raise_for_status()
            data = response.json()

            symbols = []
            for symbol_info in data['symbols']:
                if (symbol_info['status'] == 'TRADING' and 
                    symbol_info['contractType'] == 'PERPETUAL' and
                    symbol_info['symbol'].endswith('USDT')):
                    symbols.append(symbol_info['symbol'])

            return sorted(symbols)
        except requests.exceptions.RequestException as e:
            print(f"Error fetching symbols: {e}")
            return []


In [16]:
def analyze_annualized_rate_percent(days_back=90, outlier_method="iqr"):
    collector = BinanceFundingRateCollector()

    # Step 1: Get list of active USDT perpetual futures
    all_symbols = collector.get_active_futures_symbols()
    print(f"\nFound {len(all_symbols)} active USDT perpetual symbols.")
    print("First 10 symbols:", all_symbols[:10])

    # Step 2: Download and process data for all symbols
    all_data = []
    for symbol in all_symbols:
        try:
            raw_data = collector.get_all_funding_rates(symbol, days_back)
            if raw_data:
                df = collector.process_funding_data(raw_data)
                df['symbol'] = symbol
                all_data.append(df)
        except Exception as e:
            print(f"Skipping {symbol}: {e}")

    if not all_data:
        print("No data collected.")
        return None

    combined_df = pd.concat(all_data, ignore_index=True)

    # Step 3: Outlier removal based on annualizedRatePercent
    if outlier_method == "iqr":
        q1 = combined_df['annualizedRatePercent'].quantile(0.25)
        q3 = combined_df['annualizedRatePercent'].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        filtered_df = combined_df[(combined_df['annualizedRatePercent'] >= lower_bound) & (combined_df['annualizedRatePercent'] <= upper_bound)]
    else:
        filtered_df = combined_df.copy()

    # Step 4: Get latest value per symbol
    latest_df = (
        filtered_df.sort_values("fundingTime")
        .groupby("symbol")
        .tail(1)
        .set_index("symbol")
    )

    # Step 5: Compute z-score
    latest_df['zscore_annualized'] = zscore(latest_df['annualizedRatePercent'])

    # Step 6: Top/Bottom 10
    top_10 = latest_df.sort_values("zscore_annualized", ascending=False).head(10)
    bottom_10 = latest_df.sort_values("zscore_annualized", ascending=True).head(10)

    print("\n=== Top 10 Coins by Z-Score of Annualized Rate (%) ===")
    display(top_10[['annualizedRatePercent', 'zscore_annualized']])

    print("\n=== Bottom 10 Coins by Z-Score of Annualized Rate (%) ===")
    display(bottom_10[['annualizedRatePercent', 'zscore_annualized']])

    return filtered_df, latest_df, top_10, bottom_10


In [17]:
filtered_data, latest_data, top_10_z, bottom_10_z = analyze_annualized_rate_percent(days_back=90)


Found 536 active USDT perpetual symbols.
First 10 symbols: ['0GUSDT', '1000000BOBUSDT', '1000000MOGUSDT', '1000BONKUSDT', '1000CATUSDT', '1000CHEEMSUSDT', '1000FLOKIUSDT', '1000LUNCUSDT', '1000PEPEUSDT', '1000RATSUSDT']

=== Top 10 Coins by Z-Score of Annualized Rate (%) ===


,annualizedRatePercent,zscore_annualized
symbol,,
THETAUSDT,9.547305,2.243886
LTCUSDT,9.444375,2.199120
IDUSDT,9.434520,2.194834
ROSEUSDT,9.433425,2.194358
QTUMUSDT,9.319545,2.144829
XVGUSDT,9.299835,2.136257
ORDERUSDT,8.984475,1.999101
GMXUSDT,8.942865,1.981004
TLMUSDT,8.880450,1.953858



=== Bottom 10 Coins by Z-Score of Annualized Rate (%) ===


,annualizedRatePercent,zscore_annualized
symbol,,
VELODROMEUSDT,-1.322760,-2.483713
PEOPLEUSDT,-1.305240,-2.476093
0GUSDT,-1.293195,-2.470854
ARPAUSDT,-1.280055,-2.465139
IPUSDT,-1.261440,-2.457043
MELANIAUSDT,-1.257060,-2.455138
SNXUSDT,-1.227495,-2.442280
NMRUSDT,-1.225305,-2.441328
ETHFIUSDT,-1.218735,-2.438470


In [18]:
### Sorting the coins by latest APR
latest_data['annualizedRatePercent'].sort_values()

symbol
VELODROMEUSDT   -1.322760
PEOPLEUSDT      -1.305240
0GUSDT          -1.293195
ARPAUSDT        -1.280055
IPUSDT          -1.261440
                   ...   
QTUMUSDT         9.319545
ROSEUSDT         9.433425
IDUSDT           9.434520
LTCUSDT          9.444375
THETAUSDT        9.547305
Name: annualizedRatePercent, Length: 535, dtype: float64

In [19]:
filtered_data.to_excel('funding rate history.xlsx')

In [ ]:
### Identify coins which have Funding rates outside their historical +/1 SD

In [20]:
### Need to clean the data as fundingtime is slightly different for different coins (often differ by seconds)
### So consolidating by date and hour for easy comparison

def reshape_funding_data_by_hour(filtered_data: pd.DataFrame) -> pd.DataFrame:
    """
    Reshapes filtered funding data into a time-indexed DataFrame with symbols as columns
    and hourly funding rate values (annualizedRatePercent).
    """
    if filtered_data.empty:
        raise ValueError("Input DataFrame is empty.")

    # Ensure fundingTime is datetime
    filtered_data['fundingTime'] = pd.to_datetime(filtered_data['fundingTime'])

    # Round fundingTime to the hour
    filtered_data['fundingHour'] = filtered_data['fundingTime'].dt.floor('H')

    # Group by symbol and fundingHour, average in case of duplicates
    grouped = (
        filtered_data.groupby(['fundingHour', 'symbol'])['annualizedRatePercent']
        .mean()
        .reset_index()
    )

    # Pivot: rows = fundingHour, columns = symbol
    pivot_df = grouped.pivot(index='fundingHour', columns='symbol', values='annualizedRatePercent')

    # Optional: sort index and columns
    pivot_df = pivot_df.sort_index().sort_index(axis=1)

    return pivot_df

In [21]:
hourly_annualized_df = reshape_funding_data_by_hour(filtered_data)

# Display a preview
hourly_annualized_df.tail()

symbol,0GUSDT,1000000BOBUSDT,1000000MOGUSDT,1000BONKUSDT,1000CATUSDT,1000CHEEMSUSDT,1000FLOKIUSDT,1000LUNCUSDT,1000PEPEUSDT,1000RATSUSDT,...,ZETAUSDT,ZILUSDT,ZKCUSDT,ZKJUSDT,ZKUSDT,ZORAUSDT,ZRCUSDT,ZROUSDT,ZRXUSDT,币安人生USDT
fundingHour,,,,,,,,,,,,,,,,,,,,,
2025-11-12 00:00:00,NaN,5.475,5.475,NaN,NaN,5.475,NaN,NaN,0.91104,5.475,...,NaN,NaN,NaN,5.475,3.80622,NaN,5.475,NaN,NaN,5.475
2025-11-12 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-11-12 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-11-12 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-11-12 04:00:00,NaN,5.475,5.475,4.4676,5.475,5.475,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5.475,NaN,NaN,5.475,NaN,NaN,5.475


In [22]:
from scipy.stats import zscore

def compute_latest_zscores_and_highlight(df_hourly_annualized):
    """
    Compute latest z-scores for each coin vs its own history,
    and return coins with extreme high/low scores.
    """
    # Compute z-scores column-wise (axis=0), ignoring NaNs
    zscore_df = df_hourly_annualized.apply(lambda x: zscore(x.dropna()), axis=0)
    
    # Reindex back to original for proper alignment
    zscore_df = zscore_df.reindex(df_hourly_annualized.index)

    # Get the latest z-score for each coin (last non-NaN value)
    latest_zscores = zscore_df.ffill().iloc[-1]

    # Highlight coins
    high_threshold = 0.9
    low_threshold = -0.9

    top_coins = latest_zscores[latest_zscores > high_threshold].sort_values(ascending=False)
    bottom_coins = latest_zscores[latest_zscores < low_threshold].sort_values()

    print("\n=== Coins with High Z-Score ( > 0.9 ) ===")
    display(top_coins.to_frame("z-score"))

    print("\n=== Coins with Low Z-Score ( < -0.9 ) ===")
    display(bottom_coins.to_frame("z-score"))

    return latest_zscores, top_coins, bottom_coins


In [23]:
latest_z, top_z, bottom_z = compute_latest_zscores_and_highlight(hourly_annualized_df)


=== Coins with High Z-Score ( > 0.9 ) ===


,z-score
symbol,
HIPPOUSDT,2.477735
ZBTUSDT,2.443936
API3USDT,2.049253
LTCUSDT,1.845779
QTUMUSDT,1.800281
XVGUSDT,1.767533
ROSEUSDT,1.644634
ARCUSDT,1.597592
IDUSDT,1.582578



=== Coins with Low Z-Score ( < -0.9 ) ===


,z-score
symbol,
BANANAS31USDT,-11.717709
CCUSDT,-7.937254
KASUSDT,-6.746988
NMRUSDT,-6.511674
VELODROMEUSDT,-6.427129
...,...
1INCHUSDT,-0.983484
BOMEUSDT,-0.976185
DOGEUSDT,-0.941720


### Open Interest

In [2]:
import requests
import pandas as pd
import time
from datetime import datetime, timezone
from typing import List, Dict, Optional

In [3]:
class BinanceOpenInterestCollector:
    def __init__(self, sleep_between_calls: float = 0.12):
        self.base_url = "https://fapi.binance.com"
        self.session = requests.Session()
        self.session.headers.update({
            "Content-Type": "application/json",
            "User-Agent": "Mozilla/5.0"
        })
        self.sleep_between_calls = sleep_between_calls
        # endpoint for historical open interest
        self.oi_endpoint = "/futures/data/openInterestHist"

    def _safe_get(self, url: str, params: dict, timeout: int = 15) -> Optional[requests.Response]:
        """
        lightweight retry for transient failures and rate-limit (429/418).
        Keeps behaviour simple — you can replace with a more robust backoff if needed.
        """
        max_retries = 5
        backoff_base = 1.8
        attempt = 0
        while attempt <= max_retries:
            try:
                r = self.session.get(url, params=params, timeout=timeout)
            except Exception as e:
                attempt += 1
                sleep_for = backoff_base ** attempt
                print(f"[{datetime.now(timezone.utc).isoformat()}] Request exception: {e}. retry in {sleep_for:.1f}s")
                time.sleep(sleep_for)
                continue

            if 200 <= r.status_code < 300:
                return r

            if r.status_code in (429, 418):
                retry_after = r.headers.get("Retry-After")
                if retry_after:
                    try:
                        wait = float(retry_after)
                    except Exception:
                        wait = backoff_base ** (attempt + 1)
                else:
                    wait = backoff_base ** (attempt + 1)
                print(f"[{datetime.now(timezone.utc).isoformat()}] Rate limited ({r.status_code}). waiting {wait:.1f}s (attempt {attempt}/{max_retries})")
                attempt += 1
                time.sleep(wait)
                continue

            if 500 <= r.status_code < 600:
                wait = backoff_base ** (attempt + 1)
                print(f"[{datetime.now(timezone.utc).isoformat()}] Server error {r.status_code}. wait {wait:.1f}s")
                attempt += 1
                time.sleep(wait)
                continue

            # other client error: return response so caller can inspect
            print(f"[{datetime.now(timezone.utc).isoformat()}] HTTP {r.status_code}: {r.text[:400]}")
            return r

        print(f"[{datetime.now(timezone.utc).isoformat()}] Exceeded retries for {params.get('symbol')}")
        return None

    def get_open_interest_history(self,
                                  symbol: str,
                                  period: str = "1d",
                                  start_time: Optional[int] = None,
                                  end_time: Optional[int] = None,
                                  limit: int = 1000) -> List[Dict]:
        """
        Call Binance /futures/data/openInterestHist

        params:
          - symbol: e.g. "SOLUSDT"
          - period: "5m","15m","1h","4h","1d" (depends on API availability)
          - start_time, end_time: milliseconds since epoch (optional)
          - limit: number of records (max depends on endpoint; 1000 is typical)
        """
        url = self.base_url + self.oi_endpoint
        params = {"symbol": symbol, "period": period, "limit": limit}
        if start_time is not None:
            params["startTime"] = int(start_time)
        if end_time is not None:
            params["endTime"] = int(end_time)

        resp = self._safe_get(url, params)
        if resp is None:
            return []
        try:
            return resp.json()
        except Exception as e:
            print(f"Error decoding JSON for {symbol}: {e}")
            return []

    def get_all_open_interest(self, symbol: str, days_back: int = 30, period: str = "1d") -> List[Dict]:
        """
        Collect historical open interest for `symbol` going back `days_back`.
        Uses startTime/endTime windowing and paginates by advancing start to last timestamp + 1 ms.
        """
        end_time = int(time.time() * 1000)
        start_time = end_time - (days_back * 24 * 60 * 60 * 1000)
        all_data: List[Dict] = []
        current_start = start_time

        # We use a conservative batch window size (in ms): fetch up to 1000 records per call
        # The endpoint returns aggregated entries depending on `period`. We advance using last timestamp.
        while current_start < end_time:
            batch_end = min(current_start + (333 * 24 * 60 * 60 * 1000), end_time)
            batch = self.get_open_interest_history(
                symbol=symbol,
                period=period,
                start_time=current_start,
                end_time=batch_end,
                limit=1000
            )

            if not batch:
                # either no data in this window or an error occurred
                break

            all_data.extend(batch)

            # if fewer than limit returned, we likely reached end for this range
            if len(batch) < 1000:
                break

            # try to advance using the last returned timestamp; different responses may use different keys
            last = batch[-1]
            # common timestamp keys: 'timestamp', 'time', 'startTime'
            ts_keys = ['timestamp', 'time', 'startTime']
            last_ts = None
            for k in ts_keys:
                if k in last:
                    try:
                        last_ts = int(last[k])
                        break
                    except Exception:
                        pass
            if last_ts is None:
                # fallback: if response had 'sumOpenInterest' only with no timestamp we must stop to avoid infinite loop
                print(f"No timestamp in last batch for {symbol}, stopping pagination.")
                break

            # advance one ms beyond last_ts to avoid repeating
            current_start = last_ts + 1
            time.sleep(self.sleep_between_calls)

        return all_data

    def process_oi_data(self, raw_data: List[Dict], symbol: Optional[str] = None) -> pd.DataFrame:
        """
        Convert raw openInterestHist JSON list into a DataFrame with sane columns.
        Handles common field names returned by Binance's endpoint.
        """
        if not raw_data:
            return pd.DataFrame()

        df = pd.DataFrame(raw_data)

        # determine timestamp column (ms)
        ts_col = None
        for c in ['timestamp', 'time', 'startTime', 'openTime']:
            if c in df.columns:
                ts_col = c
                break

        if ts_col is not None:
            df['ts'] = pd.to_datetime(df[ts_col].astype(int), unit='ms', utc=True)
        else:
            # If there is no timestamp column, create an index-based timestamp (unlikely)
            df['ts'] = pd.NaT

        # common OI fields: sumOpenInterest, sumOpenInterestValue, avgOpenInterest
        # fallback possible: 'openInterest' (if endpoint variation)
        if 'sumOpenInterest' in df.columns:
            df['sumOpenInterest'] = pd.to_numeric(df['sumOpenInterest'], errors='coerce')
        elif 'openInterest' in df.columns:
            df['sumOpenInterest'] = pd.to_numeric(df['openInterest'], errors='coerce')

        if 'sumOpenInterestValue' in df.columns:
            df['sumOpenInterestValue'] = pd.to_numeric(df['sumOpenInterestValue'], errors='coerce')
        elif 'openInterestValue' in df.columns:
            df['sumOpenInterestValue'] = pd.to_numeric(df['openInterestValue'], errors='coerce')

        # optional average open interest
        if 'avgOpenInterest' in df.columns:
            df['avgOpenInterest'] = pd.to_numeric(df['avgOpenInterest'], errors='coerce')

        # attach symbol if provided (helpful when concatenating multiple symbols)
        if symbol:
            df['symbol'] = symbol
        elif 'symbol' not in df.columns:
            # ensure symbol column exists to keep downstream code simple
            df['symbol'] = None

        # sort and reset
        df = df.sort_values('ts').reset_index(drop=True)

        # helpful derived metrics (example): convert sumOpenInterestValue to float, compute per-contract USD if available
        # keep only useful columns
        keep_cols = [c for c in ['symbol', 'ts', 'sumOpenInterest', 'avgOpenInterest', 'sumOpenInterestValue'] if c in df.columns]
        return df[keep_cols].copy()

    def get_active_futures_symbols(self) -> List[str]:
        """
        Same code as your funding collector: returns USDT perpetuals that are TRADING.
        """
        url = f"{self.base_url}/fapi/v1/exchangeInfo"
        try:
            resp = self.session.get(url, timeout=15)
            resp.raise_for_status()
            data = resp.json()
            symbols = []
            for s in data.get('symbols', []):
                if (s.get('status') == 'TRADING'
                        and s.get('contractType') == 'PERPETUAL'
                        and s.get('symbol', '').endswith('USDT')):
                    symbols.append(s['symbol'])
            return sorted(symbols)
        except requests.exceptions.RequestException as e:
            print(f"Error fetching symbols: {e}")
            return []


def collect_all_symbols_oi(days_back: int = 30, period: str = "1d") -> (pd.DataFrame, pd.DataFrame):
    """
    High level function similar to your analyze_annualized_rate_percent:
    - find active symbols
    - fetch historical OI for each
    - return combined_df and latest_df (latest row per symbol)
    """
    collector = BinanceOpenInterestCollector()
    all_symbols = collector.get_active_futures_symbols()
    print(f"Found {len(all_symbols)} active USDT perpetual symbols. Sample: {all_symbols[:10]}")

    all_dfs = []
    for symbol in all_symbols:
        try:
            raw = collector.get_all_open_interest(symbol, days_back=days_back, period=period)
            if raw:
                df = collector.process_oi_data(raw, symbol=symbol)
                if not df.empty:
                    all_dfs.append(df)
            # polite pause between symbols (avoid bursts)
            time.sleep(0.05)
        except Exception as e:
            print(f"Skipping {symbol} due to error: {e}")

    if not all_dfs:
        print("No open interest data collected.")
        return pd.DataFrame(), pd.DataFrame()

    combined_df = pd.concat(all_dfs, ignore_index=True)

    # get latest row per symbol
    latest_df = (
        combined_df.sort_values("ts")
        .groupby("symbol", as_index=False)
        .last()
        .set_index("symbol")
    )

    return combined_df, latest_df


In [4]:
from pandas.api.types import is_datetime64_any_dtype, is_datetime64tz_dtype

if __name__ == "__main__":
    # Example usage: collect 30 days of daily OI
    combined, latest = collect_all_symbols_oi(days_back=30, period="4h")
    print("Combined shape:", combined.shape)
    print("Latest (first 10):")
    print(latest.head(10))
    
    # Ensure datetimes are timezone-naive before writing to Excel
    if 'ts' in combined.columns:
        combined['ts'] = combined['ts'].dt.tz_convert('UTC').dt.tz_localize(None)
    if 'ts' in latest.columns:
        latest['ts'] = latest['ts'].dt.tz_convert('UTC').dt.tz_localize(None)

    # Save to parquet/csv as needed
    combined.to_excel("binance_oi_combined.xlsx", index=False)
    latest.to_csv("binance_oi_latest.csv")


Found 536 active USDT perpetual symbols. Sample: ['0GUSDT', '1000000BOBUSDT', '1000000MOGUSDT', '1000BONKUSDT', '1000CATUSDT', '1000CHEEMSUSDT', '1000FLOKIUSDT', '1000LUNCUSDT', '1000PEPEUSDT', '1000RATSUSDT']
Combined shape: (94535, 4)
Latest (first 10):
                                             ts  sumOpenInterest  \
symbol                                                             
0GUSDT         1970-01-23 22:56:48.640000+00:00     1.905355e+07   
1000000BOBUSDT 1970-01-23 22:56:48.640000+00:00     3.427406e+07   
1000000MOGUSDT 1970-01-23 22:56:48.640000+00:00     3.351486e+06   
1000BONKUSDT   1970-01-23 22:56:48.640000+00:00     1.913374e+09   
1000CATUSDT    1970-01-23 22:56:48.640000+00:00     2.770911e+08   
1000CHEEMSUSDT 1970-01-23 22:56:48.640000+00:00     1.008620e+10   
1000FLOKIUSDT  1970-01-23 22:56:48.640000+00:00     1.311590e+08   
1000LUNCUSDT   1970-01-23 22:56:48.640000+00:00     8.005249e+07   
1000PEPEUSDT   1970-01-23 22:56:48.640000+00:00     1.639576e+10

In [7]:
## Getting total OI values and market share for each date
# 1) ensure ts is datetime and sort
combined = combined.copy()
combined['ts'] = pd.to_datetime(combined['ts'], utc=True)   # keep timezone if present
combined = combined.sort_values(['symbol', 'ts']).reset_index(drop=True)

# 2) tot_OI_val = total sumOpenInterestValue across all symbols for each ts
combined['tot_OI_val'] = combined.groupby('ts')['sumOpenInterestValue'].transform('sum')

# 3) %_OI_val = share (%) of each symbol's sumOpenInterestValue for that ts
#    If tot_OI_val == 0 (rare), produce 0 to avoid divide-by-zero
combined['%_OI_val'] = combined.apply(
    lambda r: 0.0 if pd.isna(r['tot_OI_val']) or r['tot_OI_val'] == 0 else (r['sumOpenInterestValue'] / r['tot_OI_val']) * 100,
    axis=1
)

# optional: round % to e.g. 6 decimal places (comment out if you prefer raw)
combined['%_OI_val'] = combined['%_OI_val'].round(6)
combined['tot_OI_val'] = combined['tot_OI_val'].astype(float)

In [8]:
# 4) Create market_share_last: last available row per symbol
# Use groupby + idxmax or last after sorting. We'll use groupby(...).last()
last_rows = (combined.sort_values('ts')
                      .groupby('symbol', as_index=False)
                      .last()[['symbol', 'ts', 'sumOpenInterest', 'sumOpenInterestValue', 'tot_OI_val', '%_OI_val']])

# name the last-date % as market_share_last_pct for clarity
last_rows = last_rows.rename(columns={
    'sumOpenInterest': 'sumOpenInterest_last',
    'sumOpenInterestValue': 'sumOpenInterestValue_last',
    'tot_OI_val': 'tot_OI_val_at_last',
    '%_OI_val': 'market_share_last_pct'   # percent on that last date
})

# 5) Compute market_share_avg using each symbol's last 29 observations
def sum_last_n_per_group(df, n=29):
    # returns DataFrame with columns ['symbol', 'last_n_sum']
    res = (df.sort_values('ts')
             .groupby('symbol')
             .apply(lambda g: g.tail(n)['sumOpenInterestValue'].sum())
             .reset_index(name='last29_sum'))
    return res

last29 = sum_last_n_per_group(combined, n=29)

# total across all coins of their last29 sums (denominator)
total_last29_sum = last29['last29_sum'].sum()
# handle case where total is 0 to avoid division by zero
if total_last29_sum == 0:
    last29['market_share_avg'] = 0.0
else:
    last29['market_share_avg'] = (last29['last29_sum'] / total_last29_sum) * 100

# merge market_share_avg into last_rows
market_share_last = last_rows.merge(last29[['symbol', 'last29_sum', 'market_share_avg']], on='symbol', how='left')

# Fill NaN market_share_avg with 0 if a symbol had no rows (shouldn't normally happen)
market_share_last['market_share_avg'] = market_share_last['market_share_avg'].fillna(0.0)

# 6) compute market_share_diff = market_share_last_pct - market_share_avg
market_share_last['market_share_last_pct'] = market_share_last['market_share_last_pct'].astype(float)
market_share_last['market_share_diff'] = market_share_last['market_share_last_pct'] - market_share_last['market_share_avg']

# optional rounding
market_share_last['market_share_last_pct'] = market_share_last['market_share_last_pct'].round(6)
market_share_last['market_share_avg'] = market_share_last['market_share_avg'].round(6)
market_share_last['market_share_diff'] = market_share_last['market_share_diff'].round(6)

# reorder columns for readability
market_share_last = market_share_last[[
    'symbol', 'ts',
    'sumOpenInterest_last', 'sumOpenInterestValue_last', 'tot_OI_val_at_last',
    'market_share_last_pct',
    'last29_sum', 'market_share_avg', 'market_share_diff'
]]


In [9]:
combined.head(5)

,symbol,ts,sumOpenInterest,sumOpenInterestValue,tot_OI_val,%_OI_val
0,0GUSDT,1969-12-25 02:56:48.640000+00:00,6660355.0,1.508059e+07,2.409650e+10,0.062584
1,0GUSDT,1969-12-25 06:56:48.640000+00:00,6601983.0,1.481211e+07,2.351452e+10,0.062991
2,0GUSDT,1969-12-25 10:56:48.640000+00:00,6689714.0,1.517294e+07,2.357094e+10,0.064371
3,0GUSDT,1969-12-25 14:56:48.640000+00:00,6684473.0,1.561055e+07,2.409773e+10,0.064780
4,0GUSDT,1969-12-25 18:56:48.640000+00:00,6733841.0,1.583732e+07,2.403357e+10,0.065897


In [10]:
market_share_last.head(5)

,symbol,ts,sumOpenInterest_last,sumOpenInterestValue_last,tot_OI_val_at_last,market_share_last_pct,last29_sum,market_share_avg,market_share_diff
0,0GUSDT,1970-01-23 22:56:48.640000+00:00,1.905355e+07,2.689218e+07,2.271071e+10,0.118412,8.374683e+08,0.123625,-0.005213
1,1000000BOBUSDT,1970-01-23 22:56:48.640000+00:00,3.427406e+07,1.534857e+06,2.271071e+10,0.006758,2.224384e+07,0.003284,0.003474
2,1000000MOGUSDT,1970-01-23 22:56:48.640000+00:00,3.351486e+06,1.193464e+06,2.271071e+10,0.005255,3.721204e+07,0.005493,-0.000238
3,1000BONKUSDT,1970-01-23 22:56:48.640000+00:00,1.913374e+09,2.376921e+07,2.271071e+10,0.104661,6.835836e+08,0.100909,0.003752
4,1000CATUSDT,1970-01-23 22:56:48.640000+00:00,2.770911e+08,1.200082e+06,2.271071e+10,0.005284,3.658135e+07,0.005400,-0.000116


In [11]:
# 7) Sort market_share_last by market_share_diff (largest to smallest)
market_share_last_sorted = market_share_last.sort_values(
    by='market_share_diff', ascending=False
).reset_index(drop=True)

# 8) Extract top and bottom 20 (symbol + value)
top_20 = market_share_last_sorted.head(20)[['symbol', 'market_share_diff']]
bottom_20 = market_share_last_sorted.tail(20)[['symbol', 'market_share_diff']]

# Convert to lists of tuples for easy further use
top_20_list = list(top_20.itertuples(index=False, name=None))
bottom_20_list = list(bottom_20.itertuples(index=False, name=None))


In [12]:
top_20_list

[('BTCUSDT', 0.788426),
 ('UNIUSDT', 0.295123),
 ('XRPUSDT', 0.085914),
 ('WLFIUSDT', 0.068061),
 ('LSKUSDT', 0.056209),
 ('BCHUSDT', 0.049367),
 ('BNBUSDT', 0.047193),
 ('ALLOUSDT', 0.031535),
 ('TRXUSDT', 0.029345),
 ('POPCATUSDT', 0.02542),
 ('ALCHUSDT', 0.019655),
 ('METUSDT', 0.018276),
 ('LTCUSDT', 0.016597),
 ('HIPPOUSDT', 0.016577),
 ('AEROUSDT', 0.015429),
 ('RESOLVUSDT', 0.015384),
 ('1000PEPEUSDT', 0.013488),
 ('MELANIAUSDT', 0.013297),
 ('JCTUSDT', 0.012125),
 ('PROMUSDT', 0.012036)]

In [13]:
bottom_20_list

[('TIAUSDT', -0.016967),
 ('GIGGLEUSDT', -0.017476),
 ('PAXGUSDT', -0.017923),
 ('ARUSDT', -0.017986),
 ('ZEREBROUSDT', -0.018568),
 ('VIRTUALUSDT', -0.01977),
 ('DASHUSDT', -0.019919),
 ('AAVEUSDT', -0.020207),
 ('ZENUSDT', -0.021343),
 ('XPLUSDT', -0.027417),
 ('AIAUSDT', -0.027474),
 ('ETCUSDT', -0.031013),
 ('DOTUSDT', -0.035193),
 ('ICPUSDT', -0.039308),
 ('PUMPUSDT', -0.039633),
 ('SUIUSDT', -0.053047),
 ('ZECUSDT', -0.087028),
 ('FILUSDT', -0.107054),
 ('NEARUSDT', -0.109134),
 ('ETHUSDT', -0.584454)]

### Liquidations

In [ ]:
### WORKING ON CODE

### Institutional vs. Retail

## Fundamental Analysis

### TVL Analysis

In [ ]:
import os
import pandas as pd
import numpy as np
import requests
from defillama2 import DefiLlama
from datetime import datetime, timedelta
import glob
from collections import Counter
import requests
from datetime import datetime
from dateutil.relativedelta import relativedelta
import time
from tqdm import tqdm

In [ ]:
###### PROTOCOL ANALYSIS ############################

In [ ]:
## This code can be run once a quarter to update the list of coins
# Create a DefiLlama instance
#obj = DefiLlama()

## Get fundamentals for all protocols
#df_protocol = obj.get_protocols_fundamentals()
#df_protocol = df_protocol.sort_values('tvl', ascending=False, ignore_index=True)
#df_protocol.to_excel('defillama_test.xlsx')

#df_prot = df_protocol[['name', 'symbol', 'category', 'tvl','change_7d']]

## Exclude Protocols with TVL less than USD 15mn
#df_prot = df_prot[df_prot['tvl'] > 15000000]

## Exclude Protocols where symbols are '-'
#df_prot_shortlst = df_prot[df_prot['symbol'] != '-']

In [ ]:
## Read the list of protocol names from excel - this includes the following filter:
## 1. TVL > 50mn
## 2. Delete protocols without coins
## 3. Keep categories: CDP, Derivatives, Dexs, Lending, Liquid Staking, Restaking, Yield, Yield Aggregator
## 4. Merge CDP & Lending into one category, Liquid Staking and Restaking together, Yield and Yield Aggregator, Derivatives and DEX together
## 5. Manually delete coins which are unknown


In [ ]:
## Note: It is possible to get the historical TVL for a protocol using the API format below for each protocol
## url = https://api.llama.fi/protocol/lido
## responses = requests.get(url)
## data = responses.json()

In [ ]:
# Path to the Excel file
protocol_file_path = r'C:\Users\jhuku\OneDrive\Documents\2. Crypto Research\Crypto Portfolio\Protocol_links.xlsx'

# Read the Excel file into a DataFrame
df_links = pd.read_excel(protocol_file_path, sheet_name = 'TVL')

# Step 2: Define the 6-month cutoff date
cutoff_date = pd.Timestamp.now() - relativedelta(months=6)

# Step 3: Initialize the combined DataFrame
combined_df = pd.DataFrame()

In [ ]:
# Iterate through the protocols with tqdm progress bar
for _, row in tqdm(df_links.iterrows(), total=len(df_links), desc="Downloading Protocol Data"):
    symbol = row['Symbol']
    url = row['Url']

    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        # Extract the TVL data
        tvl_data = data.get("tvl", [])
        df = pd.DataFrame(tvl_data)

        # Handle possible alternate column names
        if 'totalLiquidityUSD' not in df.columns and 'totalLiquidity' in df.columns:
            df.rename(columns={'totalLiquidity': 'totalLiquidityUSD'}, inplace=True)

        if df.empty or 'date' not in df.columns or 'totalLiquidityUSD' not in df.columns:
            continue

        # Convert date and filter
        df['date'] = pd.to_datetime(df['date'], unit='s')
        df = df[df['date'] >= cutoff_date]

        # Keep only date and liquidity, rename liquidity to symbol
        df = df[['date', 'totalLiquidityUSD']].rename(columns={'totalLiquidityUSD': symbol})
        df.set_index('date', inplace=True)

        # Merge into combined dataframe
        combined_df = combined_df.join(df, how='outer')

    except Exception as e:
        print(f"Error processing {symbol} ({url}): {e}")

    # Step 5: Sleep to avoid overloading API
    time.sleep(5)


In [ ]:
# Final formatting
combined_df = combined_df.sort_index().reset_index()
combined_df.set_index('date', inplace=True)

In [ ]:
# Get current date at midnight UTC
#current_date_midnight = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)

# Get yesterday's date at midnight UTC
yesterday_midnight = (datetime.now() - timedelta(days=1)).replace(hour=0, minute=0, second=0, microsecond=0)

# Filter the dataframe
#combined_df = combined_df[combined_df.index <= current_date_midnight]
combined_df = combined_df[combined_df.index <= yesterday_midnight]

In [ ]:
combined_df.tail()

In [ ]:
combined_list = combined_df.columns.to_list()

In [ ]:
# Create a dictionary from Symbol to Category for all coins
symbol_to_category = dict(zip(df_links['Symbol'], df_links['Category']))

# Lookup each symbol in combined_list using the dictionary - in case data is not downloaded for some coins
df_categories = [symbol_to_category[symbol] for symbol in combined_list if symbol in symbol_to_category]

In [ ]:
## Adding back the Theme column to the dataframe.T
df_T = combined_df.T
df_T['category'] = df_categories

In [ ]:
df_T.head(5)

In [ ]:
## Getting unique category values
category_list_short = list(set(df_categories))

In [ ]:
# Create separate datasets for each category
category_datasets = {}

for category in category_list_short:
    # Filter dataframe for the current category
    filtered_df = df_T[df_T['category'] == category]
    
    # Transpose the filtered dataframe
    transposed_df = filtered_df.T
    
    # Remove the category row (since we transposed, category is now a row)
    transposed_df = transposed_df.drop('category', axis=0)
    
    # Store in dictionary with naming convention
    category_datasets[f'{category}_df_T'] = transposed_df
    
    # Optionally, create individual variables for each category dataset
    globals()[f'{category}_df_T'] = transposed_df

# Print summary of created datasets
print("Created datasets:")
for category in category_list_short:
    dataset_name = f'{category}_df_T'
    print(f"- {dataset_name}: Shape {category_datasets[dataset_name].shape}")

In [ ]:
# Access a specific category dataset
dex_df_T = category_datasets['Dexs_df_T']  # if 'DEX' is in your category_list_short

In [ ]:
dex_df_T.tail(5)

In [ ]:
# Your dictionary of DataFrames (example naming convention: category_df_T)
# category_datasets = {
#     'dex': df_dex,
#     'lending': df_lending,
#     ...
# }

# Dictionary to store processed DataFrames
percent_tvl_dfs = {}

# Iterate through each category DataFrame
for category, df in category_datasets.items():
    df = df.copy()
    
    # Calculate total TVL for each date
    df['total_tvl'] = df.sum(axis=1)
    
    # Calculate % TVL for each coin
    percent_df = df.drop(columns='total_tvl').div(df['total_tvl'], axis=0) * 100
    
    # Store the result
    percent_tvl_dfs[category] = percent_df

# Export to Excel: each sheet is a category
with pd.ExcelWriter("category_percent_tvl.xlsx") as writer:
    for category, percent_df in percent_tvl_dfs.items():
        percent_df.to_excel(writer, sheet_name=category[:31])  # Excel sheet names limited to 31 characters


### Valuations - MarketCap/TVL

In [ ]:
import requests
import pandas as pd

In [ ]:
# Path to the Excel file
protocol_file_path = r'C:\Users\jhuku\OneDrive\Documents\2. Crypto Research\Crypto Portfolio\Protocol_links.xlsx'

# Read the Excel file into a DataFrame
df_links = pd.read_excel(protocol_file_path, sheet_name = 'Mcap')

# Step 2: Define the 6-month cutoff date
cutoff_date = pd.Timestamp.now() - relativedelta(months=6)

# Step 3: Initialize the combined DataFrame
combined_df_m = pd.DataFrame()

In [ ]:
defillama_symbols = df_links['Symbol'].to_list()

In [ ]:
coin_list = requests.get("https://api.coingecko.com/api/v3/coins/list").json()
symbol_to_id = {c['symbol']: c['id'] for c in coin_list}

In [ ]:
ids = [symbol_to_id[s.lower()] for s in defillama_symbols if s.lower() in symbol_to_id]

In [ ]:
if ids:
    params = {'vs_currency':'usd', 'ids':','.join(ids), 'order':'market_cap_desc', 'sparkline':False}
    resp = requests.get("https://api.coingecko.com/api/v3/coins/markets", params=params)
    resp.raise_for_status()
    data = resp.json()
else:
    raise ValueError("No valid CoinGecko IDs found for provided symbols.")

In [ ]:
# Convert to DataFrame
df_market_test = pd.DataFrame(data)

In [ ]:
## Keeping only the symbol and market cap columns
df_market_caps = df_market_test[['symbol','market_cap']]

In [ ]:
# Convert the 'Name' column to uppercase
df_market_caps['symbol'] = df_market_caps['symbol'].str.upper()

## Replacing the index with symbol column
df_market_caps.set_index(['symbol'], inplace=True)

In [ ]:
### Getting the latest TVL values previously calculated
TVL_latest = combined_df.tail(1)

In [ ]:
## Renaming the index of TVL df
df_renamed_axis = TVL_latest.T.rename_axis('symbol')

In [ ]:
## Merge TVL and Market Cap dfs
merged_df = pd.merge(df_renamed_axis, df_market_caps, left_index=True, right_index=True)

In [ ]:
## Renaming TVL column properly and removing any Nan values
merged_df.rename(columns={merged_df.columns[0]: 'TVL'}, inplace=True)
merged_df = merged_df.dropna()

In [ ]:
## Calculating valuation - Marketcap/TVL
merged_df['mcap/TVL'] = merged_df['market_cap']/merged_df['TVL']

In [ ]:
merged_df.sort_values(by='mcap/TVL', ascending=True)

### DefiLlama Fees Analysis

In [ ]:
url = 'https://api.llama.fi/overview/fees?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true'

In [ ]:
response = requests.get(url)
data = response.json()

In [ ]:
# Extract protocol data
protocols = data.get("protocols", [])

# Convert to DataFrame
df_fees = pd.DataFrame(protocols)

In [ ]:
# === Step 1: Split into chain_fees and protocol_fees based on protocolType ===
chain_fees = df_fees[df_fees['protocolType'] == 'chain'].copy()
protocol_fees = df_fees[df_fees['protocolType'] == 'protocol'].copy()

In [ ]:
# === Step 2: Clean 'parentProtocol' column in protocol_fees ===
protocol_fees['parentProtocol'] = protocol_fees['parentProtocol'].str.replace('parent#', '', regex=False)

In [ ]:
# === Step 3: Replace blanks in parentProtocol with slug values ===
protocol_fees['parentProtocol'] = protocol_fees['parentProtocol'].fillna(protocol_fees['slug'])

In [ ]:
# === Step 4: Keep only required columns ===
keep_cols = ['total24h', 'total7d', 'total30d', 'name', 'category', 'slug', 'parentProtocol']
chain_fees = chain_fees[keep_cols]
protocol_fees = protocol_fees[keep_cols]

In [ ]:
# Sort by parentProtocol and total30d descending (so highest total30d per group is first)
protocol_fees_sorted = protocol_fees.sort_values(
    by=["parentProtocol", "total30d"], ascending=[True, False]
)

# Group by parentProtocol and aggregate
protocol_fees_consolidated = (
    protocol_fees_sorted.groupby("parentProtocol", as_index=False).agg({
        "total24h": "sum",
        "total7d": "sum",
        "total30d": "sum",
        "name": "first",       # take from row with highest total30d (since sorted)
        "category": "first",
        "slug": "first"
    })
)

# Optional: reset index just in case
protocol_fees_consolidated.reset_index(drop=True, inplace=True)

In [ ]:
# === Step 5: Chain fees market share calculations ===
for col in ['total24h', 'total7d', 'total30d']:
    total_sum = chain_fees[col].sum()
    chain_fees[f"{col}_share"] = chain_fees[col] / total_sum

# Market share change: 7d - 30d
chain_fees['share_change'] = chain_fees['total7d_share'] - chain_fees['total30d_share']

# Top 20 chains by share_change
top20_chains = chain_fees.sort_values('share_change', ascending=False).head(20)['name'].tolist()

In [ ]:
# === Step 6: Protocol fees market share calculations ===
for col in ['total24h', 'total7d', 'total30d']:
    total_sum = protocol_fees_consolidated[col].sum()
    protocol_fees_consolidated[f"{col}_share"] = protocol_fees_consolidated[col] / total_sum

# Market share change: 7d - 30d
protocol_fees_consolidated['share_change'] = protocol_fees_consolidated['total7d_share']- protocol_fees_consolidated['total30d_share']

# Top 20 protocols by share_change (using parentProtocol)
top20_protocols = protocol_fees_consolidated.sort_values('share_change', ascending=False).head(20)['parentProtocol'].tolist()

In [ ]:
print("Top 20 Chains:", top20_chains)

In [ ]:
print("Top 20 Protocols:", top20_protocols)

In [ ]:
## Get the values for a specific Chain
input_str = "eth" ##Chain name
chain_fees_filter = chain_fees[chain_fees["name"].str.contains(input_str, case=False, na=False)]
chain_fees_filter

In [ ]:
## Get the values for a specific protocol
input_str = "jupiter" ##Protocol name
protocol_fees_filter = protocol_fees_consolidated[protocol_fees_consolidated["parentProtocol"].
                                                  str.contains(input_str, case=False, na=False)]
protocol_fees_filter

### DefiLlama Revenue Analysis

In [ ]:
url = 'https://api.llama.fi/overview/fees?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true&dataType=dailyRevenue'


In [ ]:
response = requests.get(url)
data = response.json()

In [ ]:
# Extract protocol data
protocols = data.get("protocols", [])
# Convert to DataFrame
df_rev = pd.DataFrame(protocols)

In [ ]:
# === Step 1: Split into chain_rev and protocol_rev based on protocolType ===
chain_rev = df_rev[df_rev['protocolType'] == 'chain'].copy()
protocol_rev = df_rev[df_rev['protocolType'] == 'protocol'].copy()

In [ ]:
# === Step 2: Clean 'parentProtocol' column in protocol_rev ===
protocol_rev['parentProtocol'] = protocol_rev['parentProtocol'].str.replace('parent#', '', regex=False)

In [ ]:
# === Step 3: Replace blanks in parentProtocol with slug values ===
protocol_rev['parentProtocol'] = protocol_rev['parentProtocol'].fillna(protocol_rev['slug'])

In [ ]:
# === Step 4: Keep only required columns ===
keep_cols = ['total24h', 'total7d', 'total30d', 'name', 'category', 'slug', 'parentProtocol']
chain_rev = chain_rev[keep_cols]
protocol_rev = protocol_rev[keep_cols]

In [ ]:
# Sort by parentProtocol and total30d descending (so highest total30d per group is first)
protocol_rev_sorted = protocol_rev.sort_values(
    by=["parentProtocol", "total30d"], ascending=[True, False]
)

# Group by parentProtocol and aggregate
protocol_rev_consolidated = (
    protocol_rev_sorted.groupby("parentProtocol", as_index=False).agg({
        "total24h": "sum",
        "total7d": "sum",
        "total30d": "sum",
        "name": "first",       # take from row with highest total30d (since sorted)
        "category": "first",
        "slug": "first"
    })
)

# Optional: reset index just in case
protocol_rev_consolidated.reset_index(drop=True, inplace=True)

In [ ]:
# === Step 5: Chain rev market share calculations ===
for col in ['total24h', 'total7d', 'total30d']:
    total_sum = chain_rev[col].sum()
    chain_rev[f"{col}_share"] = chain_rev[col] / total_sum
    
# Market share change: 7d - 30d
chain_rev['share_change'] = chain_rev['total7d_share'] - chain_rev['total30d_share']

# Top 20 chains by share_change
top20_chains = chain_rev.sort_values('share_change', ascending=False).head(20)['name'].tolist()

In [ ]:
# === Step 6: Protocol rev market share calculations ===
for col in ['total24h', 'total7d', 'total30d']:
    total_sum = protocol_rev_consolidated[col].sum()
    protocol_rev_consolidated[f"{col}_share"] = protocol_rev_consolidated[col] / total_sum

# Market share change: 7d - 30d
protocol_rev_consolidated['share_change'] = protocol_rev_consolidated['total7d_share'] - protocol_rev_consolidated['total30d_share']

# Top 20 protocols by share_change (using parentProtocol)
top20_protocols = protocol_rev_consolidated.sort_values('share_change', ascending=False).head(20)['parentProtocol'].tolist()

In [ ]:
print("Top 20 Chains:", top20_chains)

In [ ]:
print("Top 20 Protocols:", top20_protocols)

In [ ]:
## Get the values for a specific Chain
input_str = "bnb" ##Chain name
chain_rev_filter = chain_rev[chain_rev["name"].str.contains(input_str, case=False, na=False)]
chain_rev_filter

In [ ]:
## Get the values for a specific protocol
input_str = "jupiter" ##Protocol name
protocol_rev_filter = protocol_rev_consolidated[protocol_rev_consolidated["parentProtocol"].str.contains(input_str, case=False, na=False)]
protocol_rev_filter

### Whale vs. Retail Analysis

In [ ]:
# https://app.alphractal.com/

### Artemis/Onchain Wallet Analysis

## Options

### Tail Risk

### Premia Income

### Term Structure

#### Current 25D Implied Vol and 25D skew

In [24]:
import requests
import math
from datetime import datetime, timezone

BASE_URL = "https://www.deribit.com/api/v2"

def safe_get(url, params=None):
    r = requests.get(url, params=params)
    data = r.json()
    if "error" in data:
        raise ValueError(f"Deribit API error: {data['error']}")
    if "result" not in data:
        raise ValueError(f"Unexpected response: {data}")
    return data["result"]

def get_instruments(currency="BTC"):
    return safe_get(f"{BASE_URL}/public/get_instruments",
                    {"currency": currency, "kind": "option", "expired": "false"})

def get_index_price(currency="BTC"):
    res = safe_get(f"{BASE_URL}/public/get_index_price",
                   {"index_name": f"{currency.lower()}_usd"})
    return res["index_price"]

def get_order_book(instrument_name):
    res = safe_get(f"{BASE_URL}/public/get_order_book",
                   {"instrument_name": instrument_name})
    return res

def find_atm_iv(instruments, expiry_ts, spot):
    """Find ATM option (closest strike) and get its IV."""
    candidates = [inst for inst in instruments if inst["expiration_timestamp"] == expiry_ts]
    if not candidates:
        raise ValueError(f"No options found for expiry {expiry_ts}")
    
    closest = min(candidates, key=lambda x: abs(x["strike"] - spot))
    ob = get_order_book(closest["instrument_name"])
    iv = ob.get("mark_iv")
    
    if iv is None:
        raise ValueError(f"No mark_iv for {closest['instrument_name']}")
    
    return iv / 100  # convert % → decimal

def find_25delta_options(instruments, expiry_ts):
    """Find 25-delta put and call options and get their IVs."""
    candidates = [inst for inst in instruments if inst["expiration_timestamp"] == expiry_ts]
    if not candidates:
        raise ValueError(f"No options found for expiry {expiry_ts}")
    
    # Separate puts and calls
    puts = [inst for inst in candidates if inst["option_type"] == "put"]
    calls = [inst for inst in candidates if inst["option_type"] == "call"]
    
    # Find option closest to 25 delta for puts and calls
    # For Deribit, we look for options with greek data
    put_25d = None
    call_25d = None
    min_put_diff = float('inf')
    min_call_diff = float('inf')
    
    for put in puts:
        ob = get_order_book(put["instrument_name"])
        greeks = ob.get("greeks")
        if greeks and greeks.get("delta") is not None:
            delta = abs(greeks["delta"])
            diff = abs(delta - 0.25)
            if diff < min_put_diff:
                min_put_diff = diff
                put_25d = (put["instrument_name"], ob.get("mark_iv"))
    
    for call in calls:
        ob = get_order_book(call["instrument_name"])
        greeks = ob.get("greeks")
        if greeks and greeks.get("delta") is not None:
            delta = abs(greeks["delta"])
            diff = abs(delta - 0.25)
            if diff < min_call_diff:
                min_call_diff = diff
                call_25d = (call["instrument_name"], ob.get("mark_iv"))
    
    if put_25d is None or call_25d is None or put_25d[1] is None or call_25d[1] is None:
        raise ValueError(f"Could not find 25-delta options with valid IVs for expiry {expiry_ts}")
    
    return put_25d[1] / 100, call_25d[1] / 100  # convert % → decimal

def get_vx30(currency="BTC", target_days=30):
    """Calculate constant maturity 30-day implied volatility and 25-delta skew."""
    instruments = get_instruments(currency)
    spot = get_index_price(currency)
    
    now = datetime.now(timezone.utc).timestamp() * 1000
    expiries = sorted(list({inst["expiration_timestamp"] for inst in instruments}))
    expiries_days = [(ts, (ts - now) / 1000 / 86400) for ts in expiries]
    
    before = [e for e in expiries_days if e[1] < target_days]
    after = [e for e in expiries_days if e[1] > target_days]
    
    if not before or not after:
        raise ValueError("No expiries available around target maturity")
    
    T1, d1 = before[-1]
    T2, d2 = after[0]
    
    # Get ATM IVs
    iv1 = find_atm_iv(instruments, T1, spot)
    iv2 = find_atm_iv(instruments, T2, spot)
    
    # Get 25-delta IVs for skew calculation
    print(f"Fetching 25-delta options for {d1:.1f}d expiry...")
    put_iv1, call_iv1 = find_25delta_options(instruments, T1)
    
    print(f"Fetching 25-delta options for {d2:.1f}d expiry...")
    put_iv2, call_iv2 = find_25delta_options(instruments, T2)
    
    # Calculate 25-delta skew (Put IV - Call IV)
    skew1 = put_iv1 - call_iv1
    skew2 = put_iv2 - call_iv2
    
    # Interpolate skew to 30 days
    w = (target_days - d1) / (d2 - d1)
    skew30 = skew1 * (1 - w) + skew2 * w
    
    # Variance interpolation for ATM IV
    var1 = (iv1**2) * (d1 / 365)
    var2 = (iv2**2) * (d2 / 365)
    
    var30 = var1 * (1 - w) + var2 * w
    iv30 = math.sqrt(var30 * (365 / target_days))
    
    return iv30, iv1, d1, iv2, d2, skew30, skew1, skew2



In [25]:
# --- Run everything ---
try:
    print("Calculating BTC Volatility Metrics...")
    print("=" * 50)
    
    vx30, iv1, d1, iv2, d2, skew30, skew1, skew2 = get_vx30()
    
    print(f"\n{'CONSTANT MATURITY METRICS (30 days)':^50}")
    print("=" * 50)
    print(f"VX30 (ATM IV):        {vx30*100:>6.2f}%")
    print(f"25-Delta Skew:        {skew30*100:>6.2f}%")
    
    print(f"\n{'NEARBY EXPIRIES USED':^50}")
    print("=" * 50)
    print(f"\nShorter Expiry ({d1:.1f} days):")
    print(f"  ATM IV:             {iv1*100:>6.2f}%")
    print(f"  25-Delta Skew:      {skew1*100:>6.2f}%")
    
    print(f"\nLonger Expiry ({d2:.1f} days):")
    print(f"  ATM IV:             {iv2*100:>6.2f}%")
    print(f"  25-Delta Skew:      {skew2*100:>6.2f}%")
    
    print("\n" + "=" * 50)
    print("Note: 25-Delta Skew = 25Δ Put IV - 25Δ Call IV")
    print("      Positive skew indicates downside protection premium")
    
except Exception as e:
    print(f"Error: {e}")

Calculating BTC Volatility Metrics...
Fetching 25-delta options for 16.1d expiry...
Fetching 25-delta options for 44.1d expiry...

       CONSTANT MATURITY METRICS (30 days)        
VX30 (ATM IV):         43.35%
25-Delta Skew:          4.54%

               NEARBY EXPIRIES USED               

Shorter Expiry (16.1 days):
  ATM IV:              41.36%
  25-Delta Skew:        4.85%

Longer Expiry (44.1 days):
  ATM IV:              44.07%
  25-Delta Skew:        4.23%

Note: 25-Delta Skew = 25Δ Put IV - 25Δ Call IV
      Positive skew indicates downside protection premium


#### Historical BTC Skew

In [26]:
import requests
import math
import os
import pandas as pd
from datetime import datetime, timezone

BASE_URL = "https://www.deribit.com/api/v2"
CSV_FILE = "btc_skew_history.csv"

# ==========================================================
# --- Utility functions ---
# ==========================================================
def safe_get(url, params=None):
    r = requests.get(url, params=params)
    data = r.json()
    if "error" in data:
        raise ValueError(f"Deribit API error: {data['error']}")
    if "result" not in data:
        raise ValueError(f"Unexpected response: {data}")
    return data["result"]

def get_instruments(currency="BTC"):
    return safe_get(f"{BASE_URL}/public/get_instruments",
                    {"currency": currency, "kind": "option", "expired": "false"})

def get_index_price(currency="BTC"):
    res = safe_get(f"{BASE_URL}/public/get_index_price",
                   {"index_name": f"{currency.lower()}_usd"})
    return res["index_price"]

def get_order_book(instrument_name):
    res = safe_get(f"{BASE_URL}/public/get_order_book",
                   {"instrument_name": instrument_name})
    return res

# ==========================================================
# --- IV & skew extraction ---
# ==========================================================
def find_atm_iv(instruments, expiry_ts, spot):
    """Find ATM option (closest strike) and get its IV."""
    candidates = [inst for inst in instruments if inst["expiration_timestamp"] == expiry_ts]
    if not candidates:
        raise ValueError(f"No options found for expiry {expiry_ts}")
    
    closest = min(candidates, key=lambda x: abs(x["strike"] - spot))
    ob = get_order_book(closest["instrument_name"])
    iv = ob.get("mark_iv")
    
    if iv is None:
        raise ValueError(f"No mark_iv for {closest['instrument_name']}")
    
    return iv / 100  # convert % → decimal

def find_25delta_options(instruments, expiry_ts):
    """Find 25-delta put and call options and get their IVs."""
    candidates = [inst for inst in instruments if inst["expiration_timestamp"] == expiry_ts]
    if not candidates:
        raise ValueError(f"No options found for expiry {expiry_ts}")
    
    puts = [inst for inst in candidates if inst["option_type"] == "put"]
    calls = [inst for inst in candidates if inst["option_type"] == "call"]
    
    put_25d = None
    call_25d = None
    min_put_diff = float('inf')
    min_call_diff = float('inf')
    
    for put in puts:
        ob = get_order_book(put["instrument_name"])
        greeks = ob.get("greeks")
        if greeks and greeks.get("delta") is not None:
            delta = abs(greeks["delta"])
            diff = abs(delta - 0.25)
            if diff < min_put_diff:
                min_put_diff = diff
                put_25d = (put["instrument_name"], ob.get("mark_iv"))
    
    for call in calls:
        ob = get_order_book(call["instrument_name"])
        greeks = ob.get("greeks")
        if greeks and greeks.get("delta") is not None:
            delta = abs(greeks["delta"])
            diff = abs(delta - 0.25)
            if diff < min_call_diff:
                min_call_diff = diff
                call_25d = (call["instrument_name"], ob.get("mark_iv"))
    
    if put_25d is None or call_25d is None or put_25d[1] is None or call_25d[1] is None:
        raise ValueError(f"Could not find 25-delta options with valid IVs for expiry {expiry_ts}")
    
    return put_25d[1] / 100, call_25d[1] / 100  # convert % → decimal

# ==========================================================
# --- 30D Constant-Maturity Calculation ---
# ==========================================================
def get_vx30(currency="BTC", target_days=30):
    """Calculate constant maturity 30-day implied volatility and 25-delta skew."""
    instruments = get_instruments(currency)
    spot = get_index_price(currency)
    
    now = datetime.now(timezone.utc).timestamp() * 1000
    expiries = sorted(list({inst["expiration_timestamp"] for inst in instruments}))
    expiries_days = [(ts, (ts - now) / 1000 / 86400) for ts in expiries]
    
    before = [e for e in expiries_days if e[1] < target_days]
    after = [e for e in expiries_days if e[1] > target_days]
    
    if not before or not after:
        raise ValueError("No expiries available around target maturity")
    
    T1, d1 = before[-1]
    T2, d2 = after[0]
    
    iv1 = find_atm_iv(instruments, T1, spot)
    iv2 = find_atm_iv(instruments, T2, spot)
    
    print(f"Fetching 25-delta options for {d1:.1f}d expiry...")
    put_iv1, call_iv1 = find_25delta_options(instruments, T1)
    print(f"Fetching 25-delta options for {d2:.1f}d expiry...")
    put_iv2, call_iv2 = find_25delta_options(instruments, T2)
    
    skew1 = put_iv1 - call_iv1
    skew2 = put_iv2 - call_iv2
    
    w = (target_days - d1) / (d2 - d1)
    skew30 = skew1 * (1 - w) + skew2 * w
    
    var1 = (iv1**2) * (d1 / 365)
    var2 = (iv2**2) * (d2 / 365)
    var30 = var1 * (1 - w) + var2 * w
    iv30 = math.sqrt(var30 * (365 / target_days))
    
    return iv30, skew30

# ==========================================================
# --- Logging Function ---
# ==========================================================
def log_skew_to_csv(currency="BTC"):
    iv30, skew30 = get_vx30(currency)
    
    now = datetime.now(timezone.utc)
    row = {
        "timestamp": now.isoformat(),
        "vx30_atm_iv": iv30 * 100,
        "skew_25d": skew30 * 100
    }
    
    # Append or create CSV
    if os.path.exists(CSV_FILE):
        df = pd.read_csv(CSV_FILE)
        df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    else:
        df = pd.DataFrame([row])
    
    df.drop_duplicates(subset=["timestamp"], inplace=True)
    df.to_csv(CSV_FILE, index=False)
    
    print(f"\n--- Constant Maturity Metrics (30 days) ---")
    print(f"Timestamp:         {now.strftime('%Y-%m-%d %H:%M UTC')}")
    print(f"VX30 (ATM IV):     {iv30*100:.2f}%")
    print(f"25Δ Skew:          {skew30*100:.2f}%")
    print(f"\nLogged to: {CSV_FILE}")



In [27]:
# ==========================================================
# --- MAIN EXECUTION ---
# ==========================================================
if __name__ == "__main__":
    try:
        print("Fetching BTC 30-Day Constant Maturity Metrics...")
        log_skew_to_csv("BTC")
    except Exception as e:
        print(f"Error: {e}")


Fetching BTC 30-Day Constant Maturity Metrics...
Fetching 25-delta options for 16.1d expiry...
Fetching 25-delta options for 44.1d expiry...

--- Constant Maturity Metrics (30 days) ---
Timestamp:         2025-11-12 05:04 UTC
VX30 (ATM IV):     43.34%
25Δ Skew:          4.45%

Logged to: btc_skew_history.csv


# Risk Management

# Backtester

## Momentum Signals

## Trend Following

### Trend Following

#### Define Function/Signal

In [ ]:
import pandas as pd
import numpy as np
import random
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from typing import List, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

In [ ]:
class EnhancedTradingStrategyOptimizer:
    def __init__(self, df_h: pd.DataFrame, filtered_tickers_h: List[str]):
        """
        Initialize the optimizer with data and filtered tickers
        
        Args:
            df_h: DataFrame with datetime index and coin prices as columns
            filtered_tickers_h: List of coin tickers to analyze
        """
        self.df_h = df_h
        self.filtered_tickers_h = filtered_tickers_h
        self.df_short_h = df_h[filtered_tickers_h].copy()
        self.results_df = pd.DataFrame()
        
    def adaptive_ema(self, series: pd.Series, period: int) -> pd.Series:
        """Adaptive EMA calculation"""
        ema = series.copy()
        noise = np.zeros_like(series.values)
        
        for i in range(period, len(series)):
            sig = abs(series.iloc[i] - series.iloc[i - period])
            noise[i] = noise[i - 1] + abs(series.iloc[i] - series.iloc[i - 1]) - abs(series.iloc[i] - series.iloc[i - period])
            noise_val = noise[i] if noise[i] != 0 else 1
            efratio = sig / noise_val
            slow_end = period * 5
            fast_end = max(period / 2.0, 1)
            avg_period = ((sig / noise_val) * (slow_end - fast_end)) + fast_end
            alpha = 2.0 / (1.0 + avg_period)
            ema.iloc[i] = ema.iloc[i - 1] + alpha * (series.iloc[i] - ema.iloc[i - 1])
            
        return ema

    def jfatl(self, series: pd.Series, fatl_len: int, jma_len: int, phase: float) -> pd.Series:
        """JFATL calculation"""
        fatl = series.rolling(fatl_len).mean()
        e = 0.5 * (phase + 1)
        wma1 = fatl.rolling(jma_len).mean()
        wma2 = fatl.rolling(jma_len // 2).mean()
        return wma1 * e + wma2 * (1 - e)

    def nadaraya_watson_vectorized(self, series: pd.Series, h: int, r: float, start_regression_at_bar: int) -> pd.Series:
        """Vectorized Nadaraya-Watson Estimator"""
        n = len(series)
        smoothed = np.full(n, np.nan)
        X = np.arange(n)

        for t in range(start_regression_at_bar, n):
            indices = np.arange(0, t)
            distances = t - indices
            weights = (1 + (distances**2 / ((h**2) * 2 * r))) ** (-r)
            values = series.values[:t]
            smoothed[t] = np.sum(values * weights) / np.sum(weights)

        return pd.Series(smoothed, index=series.index)

    def generate_signals(self, coin: str, base_length: int, hold_days: int) -> Tuple[pd.Series, pd.Series]:
        """Generate buy/sell signals for a given coin, base_length, and hold_days"""
        # Parameters based on base_length
        nw_start = 150
        aema_period = 6 * base_length
        fatl_length = 6 * base_length
        jma_length = 6 * base_length
        nw_window = 6 * base_length
        nw_r = 48.0
        phase = 0.5
        
        price = self.df_short_h[coin]
        
        # Calculate indicators
        aema = self.adaptive_ema(price, aema_period)
        jfatl_val = self.jfatl(price, fatl_length, jma_length, phase)
        nw_val = self.nadaraya_watson_vectorized(price, nw_window, nw_r, nw_start)
        
        # Generate signals
        above = (price > aema) & (price > jfatl_val) & (price > nw_val)
        below = (price < aema) & (price < jfatl_val) & (price < nw_val)
        
        above_rolling = above.rolling(hold_days).sum()
        below_rolling = below.rolling(hold_days).sum()
        
        buy_signals = (above_rolling == hold_days)
        sell_signals = (below_rolling == hold_days)
        
        return buy_signals, sell_signals

    def calculate_forward_returns(self, coin: str, signals: pd.Series, signal_type: str, forward_period: int = 90) -> List[float]:
        """Calculate forward returns for given signals"""
        price = self.df_short_h[coin]
        returns = []
        
        signal_dates = signals[signals].index
        
        for date in signal_dates:
            try:
                current_idx = price.index.get_loc(date)
                if current_idx + forward_period < len(price):
                    current_price = price.iloc[current_idx]
                    future_price = price.iloc[current_idx + forward_period]
                    
                    if signal_type == 'buy':
                        ret = (future_price - current_price) / current_price
                    else:  # sell signal
                        ret = (current_price - future_price) / current_price
                    
                    returns.append(ret)
            except (KeyError, IndexError):
                continue
                
        return returns

    def run_optimization_iteration(self, base_length: int, hold_days: int, iterations: int = 500) -> Dict:
        """Run optimization for a single combination of base_length and hold_days"""
        print(f"Running optimization for base_length: {base_length}, hold_days: {hold_days}")
        
        buy_returns_all = []
        sell_returns_all = []
        total_returns_all = []
        
        for iteration in range(iterations):
            # Select random coin
            random_coin = random.choice(self.filtered_tickers_h)
            
            try:
                # Generate signals
                buy_signals, sell_signals = self.generate_signals(random_coin, base_length, hold_days)
                
                # Calculate returns
                buy_returns = self.calculate_forward_returns(random_coin, buy_signals, 'buy')
                sell_returns = self.calculate_forward_returns(random_coin, sell_signals, 'sell')
                
                # Store returns
                buy_returns_all.extend(buy_returns)
                sell_returns_all.extend(sell_returns)
                total_returns_all.extend(buy_returns + sell_returns)
                
            except Exception as e:
                print(f"Error in iteration {iteration} for coin {random_coin}: {e}")
                continue
        
        # Calculate statistics
        results = {
            'base_length': base_length,
            'hold_days': hold_days,
            'median_buy_return': np.median(buy_returns_all) if buy_returns_all else 0,
            'median_sell_return': np.median(sell_returns_all) if sell_returns_all else 0,
            'median_total_return': np.median(total_returns_all) if total_returns_all else 0,
            'num_buy_signals': len(buy_returns_all),
            'num_sell_signals': len(sell_returns_all),
            'total_signals': len(total_returns_all)
        }
        
        # Calculate additional metrics
        if total_returns_all:
            returns_array = np.array(total_returns_all)
            positive_returns = returns_array[returns_array > 0]
            negative_returns = returns_array[returns_array < 0]
            
            # Win-Loss Ratio
            win_rate = len(positive_returns) / len(returns_array) if len(returns_array) > 0 else 0
            loss_rate = len(negative_returns) / len(returns_array) if len(returns_array) > 0 else 0
            results['win_loss_ratio'] = win_rate / loss_rate if loss_rate > 0 else float('inf')
            
            # Profit Factor
            total_profits = np.sum(positive_returns) if len(positive_returns) > 0 else 0
            total_losses = abs(np.sum(negative_returns)) if len(negative_returns) > 0 else 0
            results['profit_factor'] = total_profits / total_losses if total_losses > 0 else float('inf')
            
            # Max Drawdown (simplified calculation)
            cumulative_returns = np.cumsum(returns_array)
            running_max = np.maximum.accumulate(cumulative_returns)
            drawdown = (cumulative_returns - running_max)
            results['max_drawdown'] = np.min(drawdown) if len(drawdown) > 0 else 0
        else:
            results['win_loss_ratio'] = 0
            results['profit_factor'] = 0
            results['max_drawdown'] = 0
            
        return results

    def optimize_parameters(self, base_length_range: range, hold_days_range: range, iterations: int = 500):
        """Run optimization across multiple base_length and hold_days values"""
        results = []
        
        for base_length in base_length_range:
            for hold_days in hold_days_range:
                result = self.run_optimization_iteration(base_length, hold_days, iterations)
                results.append(result)
        
        self.results_df = pd.DataFrame(results)
        return self.results_df

    def create_heatmaps(self):
        """Create heatmaps of median returns vs parameters"""
        if self.results_df.empty:
            print("No results to plot. Run optimization first.")
            return None
        
        # Create pivot tables for heatmaps
        buy_pivot = self.results_df.pivot(index='hold_days', columns='base_length', values='median_buy_return')
        sell_pivot = self.results_df.pivot(index='hold_days', columns='base_length', values='median_sell_return')
        total_pivot = self.results_df.pivot(index='hold_days', columns='base_length', values='median_total_return')
        
        # Create subplots
        fig = make_subplots(
            rows=3, cols=1,
            subplot_titles=('Buy Signal Median Returns Heatmap',
                          'Sell Signal Median Returns Heatmap',
                          'Total (Buy + Sell) Median Returns Heatmap'),
            vertical_spacing=0.1
        )
        
        # Buy returns heatmap
        fig.add_trace(
            go.Heatmap(
                z=buy_pivot.values,
                x=buy_pivot.columns,
                y=buy_pivot.index,
                colorscale='RdYlGn',
                name='Buy Returns',
                text=np.round(buy_pivot.values, 4),
                texttemplate='%{text}',
                textfont={"size": 10},
                hovertemplate='Base Length: %{x}<br>Hold Days: %{y}<br>Median Buy Return: %{z:.4f}<extra></extra>'
            ),
            row=1, col=1
        )
        
        # Sell returns heatmap
        fig.add_trace(
            go.Heatmap(
                z=sell_pivot.values,
                x=sell_pivot.columns,
                y=sell_pivot.index,
                colorscale='RdYlGn',
                name='Sell Returns',
                text=np.round(sell_pivot.values, 4),
                texttemplate='%{text}',
                textfont={"size": 10},
                hovertemplate='Base Length: %{x}<br>Hold Days: %{y}<br>Median Sell Return: %{z:.4f}<extra></extra>'
            ),
            row=2, col=1
        )
        
        # Total returns heatmap
        fig.add_trace(
            go.Heatmap(
                z=total_pivot.values,
                x=total_pivot.columns,
                y=total_pivot.index,
                colorscale='RdYlGn',
                name='Total Returns',
                text=np.round(total_pivot.values, 4),
                texttemplate='%{text}',
                textfont={"size": 10},
                hovertemplate='Base Length: %{x}<br>Hold Days: %{y}<br>Median Total Return: %{z:.4f}<extra></extra>'
            ),
            row=3, col=1
        )
        
        fig.update_layout(
            title='Median Returns Heatmaps vs Base Length and Hold Days Parameters',
            height=1200,
            width=1000,
            showlegend=False
        )
        
        # Update axis labels
        for i in range(1, 4):
            fig.update_xaxes(title_text="Base Length", row=i, col=1)
            fig.update_yaxes(title_text="Hold Days", row=i, col=1)
        
        return fig

    def create_performance_line_charts(self):
        """Create interactive line charts for performance metrics"""
        if self.results_df.empty:
            print("No results to plot. Run optimization first.")
            return None
        
        # Handle infinite values for better plotting
        chart_data = self.results_df.copy()
        chart_data['profit_factor'] = chart_data['profit_factor'].replace([np.inf, -np.inf], np.nan)
        chart_data['win_loss_ratio'] = chart_data['win_loss_ratio'].replace([np.inf, -np.inf], np.nan)
        
        # Cap extremely high values for better visualization
        chart_data['profit_factor'] = np.clip(chart_data['profit_factor'], 0, 10)
        chart_data['win_loss_ratio'] = np.clip(chart_data['win_loss_ratio'], 0, 10)
        
        # Create parameter combination for x-axis
        chart_data['param_combo'] = chart_data['base_length'].astype(str) + '_' + chart_data['hold_days'].astype(str)
        chart_data = chart_data.sort_values(['base_length', 'hold_days'])
        
        # Create subplots - 3 charts in separate rows
        fig = make_subplots(
            rows=3, cols=1,
            subplot_titles=('Profit Factor vs Parameters', 
                          'Win-Loss Ratio vs Parameters',
                          'Max Drawdown vs Parameters'),
            vertical_spacing=0.1
        )
        
        # Profit Factor
        fig.add_trace(
            go.Scatter(x=list(range(len(chart_data))), y=chart_data['profit_factor'],
                      mode='lines+markers', name='Profit Factor', line=dict(color='green'),
                      text=chart_data['param_combo'],
                      hovertemplate='Base Length_Hold Days: %{text}<br>Profit Factor: %{y:.3f}<extra></extra>'),
            row=1, col=1
        )
        
        # Win-Loss Ratio
        fig.add_trace(
            go.Scatter(x=list(range(len(chart_data))), y=chart_data['win_loss_ratio'],
                      mode='lines+markers', name='Win-Loss Ratio', line=dict(color='blue'),
                      text=chart_data['param_combo'],
                      hovertemplate='Base Length_Hold Days: %{text}<br>Win-Loss Ratio: %{y:.3f}<extra></extra>'),
            row=2, col=1
        )
        
        # Max Drawdown (make positive for better interpretation)
        fig.add_trace(
            go.Scatter(x=list(range(len(chart_data))), y=-chart_data['max_drawdown'],
                      mode='lines+markers', name='Max Drawdown', line=dict(color='red'),
                      text=chart_data['param_combo'],
                      hovertemplate='Base Length_Hold Days: %{text}<br>Max Drawdown: %{y:.3f}<extra></extra>'),
            row=3, col=1
        )
        
        fig.update_layout(
            title='Performance Metrics vs Parameter Combinations',
            height=1200,
            width=1000,
            showlegend=False
        )
        
        # Update x-axis labels
        for i in range(1, 4):
            fig.update_xaxes(title_text="Parameter Combination Index", row=i, col=1)
        
        # Update y-axis labels
        fig.update_yaxes(title_text="Profit Factor", row=1, col=1)
        fig.update_yaxes(title_text="Win-Loss Ratio", row=2, col=1)
        fig.update_yaxes(title_text="Max Drawdown", row=3, col=1)
        
        return fig

    def analyze_specific_coins(self, coin_symbols: List[str], base_length: int = 14, hold_days: int = 18):
        """Analyze specific coins and create interactive charts"""
        charts = {}
        
        for coin in coin_symbols:
            if coin not in self.df_short_h.columns:
                print(f"Coin {coin} not found in data")
                continue
                
            # Generate signals
            buy_signals, sell_signals = self.generate_signals(coin, base_length, hold_days)
            price = self.df_short_h[coin]
            
            # Calculate returns for each signal
            buy_signal_dates = buy_signals[buy_signals].index
            sell_signal_dates = sell_signals[sell_signals].index
            
            buy_returns = []
            sell_returns = []
            buy_dates_with_returns = []
            sell_dates_with_returns = []
            
            # Calculate returns for buy signals
            for date in buy_signal_dates:
                returns = self.calculate_forward_returns(coin, pd.Series([True], index=[date]), 'buy')
                if returns:
                    buy_returns.append(returns[0])
                    buy_dates_with_returns.append(date)
            
            # Calculate returns for sell signals
            for date in sell_signal_dates:
                returns = self.calculate_forward_returns(coin, pd.Series([True], index=[date]), 'sell')
                if returns:
                    sell_returns.append(returns[0])
                    sell_dates_with_returns.append(date)
            
            # Create subplots for this coin
            fig = make_subplots(
                rows=2, cols=1,
                subplot_titles=(f'{coin} Price with Buy/Sell Signals', 
                              f'{coin} Signal Returns Over Time'),
                specs=[[{"secondary_y": False}], [{"secondary_y": False}]],
                vertical_spacing=0.15
            )
            
            # Price chart with signals
            fig.add_trace(
                go.Scatter(x=price.index, y=price.values,
                          mode='lines', name=f'{coin} Price', line=dict(color='black')),
                row=1, col=1
            )
            
            # Buy signals
            if len(buy_signal_dates) > 0:
                buy_prices = [price.loc[date] for date in buy_signal_dates]
                fig.add_trace(
                    go.Scatter(x=buy_signal_dates, y=buy_prices,
                              mode='markers', name='Buy Signals', 
                              marker=dict(color='green', size=6, symbol='circle'),
                              hovertemplate='Date: %{x}<br>Price: %{y:.4f}<br>Signal: Buy<extra></extra>'),
                    row=1, col=1
                )
            
            # Sell signals
            if len(sell_signal_dates) > 0:
                sell_prices = [price.loc[date] for date in sell_signal_dates]
                fig.add_trace(
                    go.Scatter(x=sell_signal_dates, y=sell_prices,
                              mode='markers', name='Sell Signals',
                              marker=dict(color='red', size=6, symbol='circle'),
                              hovertemplate='Date: %{x}<br>Price: %{y:.4f}<br>Signal: Sell<extra></extra>'),
                    row=1, col=1
                )
            
            # Returns chart
            if buy_returns:
                fig.add_trace(
                    go.Scatter(x=buy_dates_with_returns, y=buy_returns,
                              mode='markers', name='Buy Returns',
                              marker=dict(color='green', size=8, symbol='circle'),
                              hovertemplate='Date: %{x}<br>Return: %{y:.4f}<br>Signal: Buy<extra></extra>'),
                    row=2, col=1
                )
            
            if sell_returns:
                fig.add_trace(
                    go.Scatter(x=sell_dates_with_returns, y=sell_returns,
                              mode='markers', name='Sell Returns',
                              marker=dict(color='red', size=8, symbol='circle'),
                              hovertemplate='Date: %{x}<br>Return: %{y:.4f}<br>Signal: Sell<extra></extra>'),
                    row=2, col=1
                )
            
            # Add zero line for returns
            fig.add_hline(y=0, line_dash="dash", line_color="gray", row=2, col=1)
            
            fig.update_layout(
                title=f'{coin} Trading Analysis (Base Length: {base_length}, Hold Days: {hold_days})',
                height=800,
                width=1200,
                showlegend=True
            )
            
            fig.update_xaxes(title_text="Date", row=1, col=1)
            fig.update_xaxes(title_text="Date", row=2, col=1)
            fig.update_yaxes(title_text="Price", row=1, col=1)
            fig.update_yaxes(title_text="Return", row=2, col=1)
            
            charts[coin] = fig
        
        return charts

    def get_best_parameters(self, metric='median_total_return'):
        """Get the best parameter combination based on specified metric"""
        if self.results_df.empty:
            print("No results available. Run optimization first.")
            return None
        
        if metric not in self.results_df.columns:
            print(f"Metric '{metric}' not found in results.")
            return None
        
        best_idx = self.results_df[metric].idxmax()
        best_params = self.results_df.loc[best_idx]
        
        print(f"Best parameters based on {metric}:")
        print(f"Base Length: {best_params['base_length']}")
        print(f"Hold Days: {best_params['hold_days']}")
        print(f"{metric}: {best_params[metric]:.6f}")
        
        return best_params



#### Optimization & Charting

In [ ]:
# 1. Initialize the optimizer
optimizer = EnhancedTradingStrategyOptimizer(df_h, filtered_tickers_h)

In [ ]:
# 2. Run parameter optimization
base_length_range = range(8, 25, 2)  # 8, 10, 12, 14, 16, 18, 20, 22, 24
hold_days_range = range(3, 18, 2)    # 3, 5, 7, 9, 11, 13, 15, 17
results_df = optimizer.optimize_parameters(base_length_range, hold_days_range, iterations=500)


In [ ]:
# 3. Create and display heatmaps
heatmap_fig = optimizer.create_heatmaps()
heatmap_fig.show()

In [ ]:
# 4. Create and display performance line charts
line_charts_fig = optimizer.create_performance_line_charts()
line_charts_fig.show()

In [ ]:
# 5. Find best parameters
best_params = optimizer.get_best_parameters('median_total_return')

In [ ]:
# 6. Analyze specific coins
coin_symbols = ['WOO']  # Replace with your actual coin symbols
coin_charts = optimizer.analyze_specific_coins(coin_symbols, base_length=16, hold_days=6)

# Display charts for each coin
for coin, chart in coin_charts.items():
    chart.show()

### Return-to-Trend

## Long/Short

## Value Buys

### Fresh Momentum

#### Define Function/signal

In [ ]:
def compute_fresh_momentum_series(prices_df, lookback_days=42, holding_period=30):
    """
    Computes fresh momentum for each coin across all dates.
    
    Parameters:
    - prices_df: DataFrame with datetime index and coin tickers as columns
    - lookback_days: Lookback period for calculating return
    - holding_period: Holding period for forward return
    
    Returns:
    - DataFrame of fresh momentum values for each date and coin
    """
    # Initialize empty DataFrame to store fresh momentum values
    fm_df = pd.DataFrame(index=prices_df.index, columns=prices_df.columns, dtype='float64')

    # Loop through each coin
    for coin in prices_df.columns:
        close_prices = prices_df[coin]
        
        # Calculate forward return over holding period, offset by lookback
        returns = close_prices.pct_change(periods=lookback_days).shift(-holding_period)

        # Clip to get positive and negative returns
        pos_return = returns.clip(lower=0)
        neg_return = returns.clip(upper=0)
        
        # Final fresh momentum = positive minus negative return
        fm = pos_return - neg_return
        
        fm_df[coin] = fm

    return fm_df

In [ ]:
fresh_momentum_all = compute_fresh_momentum_series(df_short_h[filtered_tickers_h])

In [ ]:
# Remove rows where all coin momentum values are NaN
fresh_momentum_all_cleaned = fresh_momentum_all.dropna(how='all')

#### Quintile Analysis

In [ ]:
# Parameters
num_iterations = 500
forward_periods = 120  # 30 days of 4-hour intervals
num_deciles = 10

In [ ]:
# Initialize structure to store decile returns
decile_returns = {i: [] for i in range(1, num_deciles + 1)}

# Drop rows where all values are NaN (if not already cleaned)
fresh_momentum_all_cleaned = fresh_momentum_all_cleaned.dropna(how='all')

# Use index values that have sufficient forward data
valid_indices = fresh_momentum_all_cleaned.index[:-forward_periods]

In [ ]:
# Perform 500 random iterations
for _ in range(num_iterations):
    # Randomly choose a date index
    rand_date = np.random.choice(valid_indices)
    mom_row = fresh_momentum_all_cleaned.loc[rand_date].dropna()

    # Skip if not enough coins to form all deciles
    if len(mom_row) < num_deciles:
        continue

    # Sort coins by fresh momentum (descending)
    sorted_coins = mom_row.sort_values(ascending=False)

    # Size of each decile
    decile_size = len(sorted_coins) // num_deciles

    # Create deciles and calculate forward returns
    for i in range(num_deciles):
        decile_coins = sorted_coins.iloc[i * decile_size : (i + 1) * decile_size].index

        try:
            # Get current and future prices
            current_prices = df_short_h.loc[rand_date, decile_coins]
            future_index = df_short_h.index.get_loc(rand_date) + forward_periods
            future_date = df_short_h.index[future_index]
            future_prices = df_short_h.loc[future_date, decile_coins]

            # Compute average return of decile
            forward_returns = (future_prices - current_prices) / current_prices
            avg_return = forward_returns.mean()

            # Save the result
            decile_returns[i + 1].append(avg_return)
        except (KeyError, IndexError):
            continue  # Skip iteration if future date is out of bounds


In [ ]:
# Compute average returns per decile
avg_decile_returns = {
    decile: np.mean(returns) if returns else np.nan
    for decile, returns in decile_returns.items()
}

In [ ]:
# Plot histogram of average returns
plt.figure(figsize=(10, 6))
plt.bar(avg_decile_returns.keys(), avg_decile_returns.values(), color='skyblue')
plt.xlabel('Decile (1 = Highest Momentum)')
plt.ylabel('Average Forward 30-Day Return')
plt.title('Average Forward Returns by Fresh Momentum Decile (500 Samples)')
plt.xticks(range(1, num_deciles + 1))
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

#### Optimization & Charting

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def compute_fresh_momentum_series(prices_df, lookback_days=42, holding_period=30):
    """
    Computes fresh momentum for each coin across all dates.
    
    Parameters:
    - prices_df: DataFrame with datetime index and coin tickers as columns
    - lookback_days: Lookback period for calculating return
    - holding_period: Holding period for forward return
    
    Returns:
    - DataFrame of fresh momentum values for each date and coin
    """
    # Initialize empty DataFrame to store fresh momentum values
    fm_df = pd.DataFrame(index=prices_df.index, columns=prices_df.columns, dtype='float64')
    
    # Loop through each coin
    for coin in prices_df.columns:
        close_prices = prices_df[coin]
        
        # Calculate forward return over holding period, offset by lookback
        returns = close_prices.pct_change(periods=lookback_days).shift(-holding_period)
        # Clip to get positive and negative returns
        pos_return = returns.clip(lower=0)
        neg_return = returns.clip(upper=0)
        
        # Final fresh momentum = positive minus negative return
        fm = pos_return - neg_return
        
        fm_df[coin] = fm
    
    return fm_df

def calculate_forward_returns(prices_df, period=90):
    """
    Calculate forward returns for given period
    """
    return prices_df.pct_change(periods=period).shift(-period)

def create_deciles(momentum_series):
    """
    Create deciles based on momentum values
    Returns dict with decile number as key and coin list as value
    """
    # Remove NaN values
    valid_momentum = momentum_series.dropna()
    
    if len(valid_momentum) == 0:
        return {}
    
    # Sort by momentum values in descending order
    sorted_momentum = valid_momentum.sort_values(ascending=False)
    
    # Create deciles
    deciles = {}
    n_coins = len(sorted_momentum)
    coins_per_decile = max(1, n_coins // 10)  # At least 1 coin per decile
    
    for i in range(10):
        start_idx = i * coins_per_decile
        end_idx = min((i + 1) * coins_per_decile, n_coins)
        
        if start_idx < n_coins:
            decile_coins = sorted_momentum.iloc[start_idx:end_idx].index.tolist()
            deciles[i + 1] = decile_coins
    
    return deciles

def calculate_performance_metrics(returns_series):
    """
    Calculate performance metrics for a return series
    """
    returns_series = returns_series.dropna()
    
    if len(returns_series) == 0:
        return {
            'total_return': 0,
            'profit_factor': 1,
            'win_loss_ratio': 1,
            'max_drawdown': 0
        }
    
    # Total return
    total_return = (1 + returns_series).prod() - 1
    
    # Win/Loss metrics
    positive_returns = returns_series[returns_series > 0]
    negative_returns = returns_series[returns_series < 0]
    
    profit_factor = abs(positive_returns.sum() / negative_returns.sum()) if negative_returns.sum() != 0 else np.inf
    win_loss_ratio = len(positive_returns) / len(negative_returns) if len(negative_returns) > 0 else np.inf
    
    # Max drawdown
    cumulative_returns = (1 + returns_series).cumprod()
    running_max = cumulative_returns.cummax()
    drawdown = (cumulative_returns - running_max) / running_max
    max_drawdown = drawdown.min()
    
    return {
        'total_return': total_return,
        'profit_factor': profit_factor,
        'win_loss_ratio': win_loss_ratio,
        'max_drawdown': abs(max_drawdown)
    }

def run_parameter_optimization(prices_df, filtered_tickers, 
                             lookback_range=(10, 100, 10), 
                             holding_range=(10, 60, 10),
                             n_iterations=500):
    """
    Run parameter optimization for fresh momentum strategy
    
    Parameters:
    - prices_df: DataFrame with price data
    - filtered_tickers: List of tickers to analyze
    - lookback_range: (start, stop, step) for lookback_days parameter
    - holding_range: (start, stop, step) for holding_period parameter
    - n_iterations: Number of random samples per parameter combination
    """
    
    # Generate parameter combinations
    lookback_values = range(lookback_range[0], lookback_range[1] + 1, lookback_range[2])
    holding_values = range(holding_range[0], holding_range[1] + 1, holding_range[2])
    
    # Initialize results storage
    performance_metrics = []
    
    total_combinations = len(lookback_values) * len(holding_values)
    
    print(f"Testing {total_combinations} parameter combinations with {n_iterations} iterations each...")
    
    for lookback_days in tqdm(lookback_values, desc="Lookback Days"):
        for holding_period in tqdm(holding_values, desc="Holding Period", leave=False):
            
            # Calculate fresh momentum for current parameters
            fm_df = compute_fresh_momentum_series(prices_df[filtered_tickers], 
                                                lookback_days=lookback_days, 
                                                holding_period=holding_period)
            
            # Calculate forward returns (15 days = 90 periods for 4-hourly data)
            forward_returns = calculate_forward_returns(prices_df[filtered_tickers], period=90)
            
            # Storage for iteration results
            iteration_returns = []
            
            # Run multiple iterations
            for iteration in range(n_iterations):
                # Get valid dates where we have both momentum and forward return data
                valid_dates = fm_df.index.intersection(forward_returns.index)
                valid_dates = valid_dates[~fm_df.loc[valid_dates].isna().all(axis=1)]
                valid_dates = valid_dates[~forward_returns.loc[valid_dates].isna().all(axis=1)]
                
                if len(valid_dates) == 0:
                    continue
                
                # Randomly select a date
                random_date = np.random.choice(valid_dates)
                
                # Get momentum values for that date
                momentum_values = fm_df.loc[random_date]
                
                # Create deciles
                deciles = create_deciles(momentum_values)
                
                if 1 not in deciles:  # Top decile
                    continue
                
                # Get forward returns for top decile coins
                top_decile_coins = deciles[1]
                top_decile_returns = forward_returns.loc[random_date, top_decile_coins]
                
                # Calculate average return for top decile
                avg_return = top_decile_returns.mean()
                if not np.isnan(avg_return):
                    iteration_returns.append(avg_return)
            
            # Calculate average return across iterations
            if len(iteration_returns) > 0:
                avg_return_across_iterations = np.mean(iteration_returns)
                
                # Calculate performance metrics
                returns_series = pd.Series(iteration_returns)
                metrics = calculate_performance_metrics(returns_series)
                
                # Store results
                result_row = {
                    'lookback_days': lookback_days,
                    'holding_period': holding_period,
                    'avg_return': avg_return_across_iterations,
                    'total_return': metrics['total_return'],
                    'profit_factor': metrics['profit_factor'],
                    'win_loss_ratio': metrics['win_loss_ratio'],
                    'max_drawdown': metrics['max_drawdown']
                }
                
                performance_metrics.append(result_row)
    
    # Convert to DataFrame
    results_df = pd.DataFrame(performance_metrics)
    
    return results_df

def create_interactive_heatmap(results_df):
    """
    Create interactive heatmap of average returns
    """
    pivot_returns = results_df.pivot(index='holding_period', columns='lookback_days', values='avg_return')
    
    fig = go.Figure(data=go.Heatmap(
        z=pivot_returns.values,
        x=pivot_returns.columns,
        y=pivot_returns.index,
        colorscale='RdYlGn',
        zmid=0,
        text=pivot_returns.values,
        texttemplate="%{text:.4f}",
        textfont={"size": 10},
        hoverongaps=False,
        hovertemplate='<b>Lookback Days</b>: %{x}<br>' +
                     '<b>Holding Period</b>: %{y}<br>' +
                     '<b>Average Return</b>: %{z:.4f}<extra></extra>'
    ))
    
    fig.update_layout(
        title='Average Returns Heatmap (Top Decile) - Interactive',
        xaxis_title='Lookback Days',
        yaxis_title='Holding Period',
        width=900,
        height=700,
        font=dict(size=12)
    )
    
    fig.show()

def create_interactive_line_charts(results_df):
    """
    Create interactive line charts for all performance metrics in single column layout
    """
    fig = make_subplots(
        rows=4, cols=1,  # Changed from 2x2 to 4x1
        subplot_titles=('Profit Factor vs Lookback Days', 'Win/Loss Ratio vs Lookback Days', 
                       'Max Drawdown vs Lookback Days', 'Average Return vs Lookback Days'),
        vertical_spacing=0.08,  # Reduced spacing since we have more rows
        horizontal_spacing=0.08
    )
    
    colors = px.colors.qualitative.Set1
    
    for i, holding_period in enumerate(sorted(results_df['holding_period'].unique())):
        subset = results_df[results_df['holding_period'] == holding_period]
        color = colors[i % len(colors)]
        
        # Profit Factor - Row 1
        fig.add_trace(
            go.Scatter(x=subset['lookback_days'], y=subset['profit_factor'],
                      mode='lines+markers', name=f'Holding={holding_period}',
                      line=dict(color=color, width=3), 
                      marker=dict(size=8),
                      legendgroup=f'group{i}',
                      hovertemplate=f'<b>Lookback</b>: %{{x}}<br>' +
                                   f'<b>Holding</b>: {holding_period}<br>' +
                                   '<b>Profit Factor</b>: %{y:.4f}<extra></extra>'),
            row=1, col=1
        )
        
        # Win/Loss Ratio - Row 2
        fig.add_trace(
            go.Scatter(x=subset['lookback_days'], y=subset['win_loss_ratio'],
                      mode='lines+markers', name=f'Holding={holding_period}',
                      line=dict(color=color, width=3), 
                      marker=dict(size=8),
                      legendgroup=f'group{i}', showlegend=False,
                      hovertemplate=f'<b>Lookback</b>: %{{x}}<br>' +
                                   f'<b>Holding</b>: {holding_period}<br>' +
                                   '<b>Win/Loss Ratio</b>: %{y:.4f}<extra></extra>'),
            row=2, col=1
        )
        
        # Max Drawdown - Row 3
        fig.add_trace(
            go.Scatter(x=subset['lookback_days'], y=subset['max_drawdown'],
                      mode='lines+markers', name=f'Holding={holding_period}',
                      line=dict(color=color, width=3), 
                      marker=dict(size=8),
                      legendgroup=f'group{i}', showlegend=False,
                      hovertemplate=f'<b>Lookback</b>: %{{x}}<br>' +
                                   f'<b>Holding</b>: {holding_period}<br>' +
                                   '<b>Max Drawdown</b>: %{y:.4f}<extra></extra>'),
            row=3, col=1
        )
        
        # Average Return - Row 4
        fig.add_trace(
            go.Scatter(x=subset['lookback_days'], y=subset['avg_return'],
                      mode='lines+markers', name=f'Holding={holding_period}',
                      line=dict(color=color, width=3), 
                      marker=dict(size=8),
                      legendgroup=f'group{i}', showlegend=False,
                      hovertemplate=f'<b>Lookback</b>: %{{x}}<br>' +
                                   f'<b>Holding</b>: {holding_period}<br>' +
                                   '<b>Average Return</b>: %{y:.4f}<extra></extra>'),
            row=4, col=1
        )
    
    # Update x-axis titles for all subplots
    fig.update_xaxes(title_text="Lookback Days", row=1, col=1)
    fig.update_xaxes(title_text="Lookback Days", row=2, col=1)
    fig.update_xaxes(title_text="Lookback Days", row=3, col=1)
    fig.update_xaxes(title_text="Lookback Days", row=4, col=1)
    
    # Update y-axis titles
    fig.update_yaxes(title_text="Profit Factor", row=1, col=1)
    fig.update_yaxes(title_text="Win/Loss Ratio", row=2, col=1)
    fig.update_yaxes(title_text="Max Drawdown", row=3, col=1)
    fig.update_yaxes(title_text="Average Return", row=4, col=1)
    
    fig.update_layout(
        title='Performance Metrics vs Parameters - Interactive (Single Column)',
        height=1200,  # Increased height to accommodate 4 rows
        width=1000,   # Reduced width since we only have 1 column
        hovermode='closest',
        font=dict(size=12)
    )
    
    fig.show()

def create_individual_metric_charts(results_df):
    """
    Create separate interactive charts for each metric with better zoom control
    """
    metrics = [
        ('avg_return', 'Average Return'),
        ('profit_factor', 'Profit Factor'), 
        ('win_loss_ratio', 'Win/Loss Ratio'),
        ('max_drawdown', 'Max Drawdown')
    ]
    
    colors = px.colors.qualitative.Set1
    
    for metric_col, metric_name in metrics:
        fig = go.Figure()
        
        for i, holding_period in enumerate(sorted(results_df['holding_period'].unique())):
            subset = results_df[results_df['holding_period'] == holding_period]
            
            fig.add_trace(go.Scatter(
                x=subset['lookback_days'],
                y=subset[metric_col],
                mode='lines+markers',
                name=f'Holding={holding_period}',
                line=dict(color=colors[i % len(colors)], width=4),
                marker=dict(size=10),
                hovertemplate=f'<b>Lookback Days</b>: %{{x}}<br>' +
                            f'<b>Holding Period</b>: {holding_period}<br>' +
                            f'<b>{metric_name}</b>: %{{y:.4f}}<extra></extra>'
            ))
        
        fig.update_layout(
            title=f'{metric_name} vs Lookback Days - Interactive Zoom',
            xaxis_title='Lookback Days',
            yaxis_title=metric_name,
            hovermode='closest',
            width=1000,
            height=600,
            showlegend=True,
            font=dict(size=12)
        )
        
        fig.show()

def create_visualizations(results_df):
    """
    Create all interactive visualizations
    """
    if results_df.empty:
        print("No results to visualize")
        return
    
    print("\n=== CREATING INTERACTIVE VISUALIZATIONS ===")
    print("Features available:")
    print("1. Click and drag to zoom into specific areas")
    print("2. Double-click to reset zoom")
    print("3. Hover over points/areas to see exact values")
    print("4. Use toolbar to pan, zoom, and save charts")
    print("5. Click legend items to show/hide specific series\n")
    
    # Create heatmap
    print("Creating interactive heatmap...")
    create_interactive_heatmap(results_df)
    
    # Create combined line charts
    print("Creating combined line charts...")
    create_interactive_line_charts(results_df)
    
    # Create individual metric charts for detailed analysis
    print("Creating individual metric charts...")
    create_individual_metric_charts(results_df)

def find_optimal_parameters(results_df):
    """
    Find optimal parameters based on different criteria
    """
    if results_df.empty:
        print("No results to analyze")
        return
    
    print("=== OPTIMAL PARAMETER ANALYSIS ===\n")
    
    # Best by average return
    best_return_idx = results_df['avg_return'].idxmax()
    best_return = results_df.loc[best_return_idx]
    print(f"Best Average Return: {best_return['avg_return']:.4f}")
    print(f"Parameters: Lookback={best_return['lookback_days']}, Holding={best_return['holding_period']}")
    print(f"Profit Factor: {best_return['profit_factor']:.4f}")
    print(f"Win/Loss Ratio: {best_return['win_loss_ratio']:.4f}")
    print(f"Max Drawdown: {best_return['max_drawdown']:.4f}\n")
    
    # Best by profit factor
    best_pf_idx = results_df['profit_factor'].idxmax()
    best_pf = results_df.loc[best_pf_idx]
    print(f"Best Profit Factor: {best_pf['profit_factor']:.4f}")
    print(f"Parameters: Lookback={best_pf['lookback_days']}, Holding={best_pf['holding_period']}")
    print(f"Average Return: {best_pf['avg_return']:.4f}")
    print(f"Win/Loss Ratio: {best_pf['win_loss_ratio']:.4f}")
    print(f"Max Drawdown: {best_pf['max_drawdown']:.4f}\n")
    
    # Best by Sharpe-like ratio (return/drawdown)
    results_df['return_drawdown_ratio'] = results_df['avg_return'] / (results_df['max_drawdown'] + 1e-6)
    best_sharpe_idx = results_df['return_drawdown_ratio'].idxmax()
    best_sharpe = results_df.loc[best_sharpe_idx]
    print(f"Best Return/Drawdown Ratio: {best_sharpe['return_drawdown_ratio']:.4f}")
    print(f"Parameters: Lookback={best_sharpe['lookback_days']}, Holding={best_sharpe['holding_period']}")
    print(f"Average Return: {best_sharpe['avg_return']:.4f}")
    print(f"Profit Factor: {best_sharpe['profit_factor']:.4f}")
    print(f"Max Drawdown: {best_sharpe['max_drawdown']:.4f}\n")



In [ ]:
# Run the parameter optimization
results_df = run_parameter_optimization(
    prices_df=df_short_h, 
    filtered_tickers=filtered_tickers_h,
    lookback_range=(18, 180, 6),  # Test lookback from 3 to 30 days in steps of 1 day
    holding_range=(18, 180, 12),    # Test holding from 3 to 30 days in steps of 2 days
    n_iterations=500
)

In [ ]:
# Create all interactive visualizations
create_visualizations(results_df)

In [ ]:
# Find optimal parameters
find_optimal_parameters(results_df)

## Technical Positioning

## Fundamental Analysis

## Options

# TEST